<h1>Numerical Methods in Python</h1>

This Jupyter notebook serves as supplementary material to the Python code from the book [Numerical Methods for Scientific Computing](https://nbviewer.jupyter.org/github/nmfsc/book/blob/main/NMFSC.pdf).  These code snippets are minimal working toy algorithms meant to better understand the mathematics that goes into them. They are tools for tinkering and learning. Play with them and have fun. And, perhaps you can repurpose a few of them. In practice, use real packages. By-and-large, the snippets are verbatim from the book with the an occasional explicit plot statement or variable declaration.  The notebook is designed to be nonlinear⁠—you can jump around.  We'll load the following packages which are used throughout this notebook:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.linalg as la
import scipy.sparse as sp
bucket = "https://raw.githubusercontent.com/nmfsc/data/master/"

We can set the output of matplotlib in Jupyter to produce SVG, which will result in higher quality plots than the default PNG. Rendering a SVG is a little slower than a PNG and it can produce noticeable jerkiness in ipywidgets animations.  To switch back to the default PNG, use the command `set_matplotlib_formats('png')`.

In [ ]:
from IPython.display import set_matplotlib_formats
%matplotlib inline
set_matplotlib_formats('svg')

<h1>Notebook Contents</h1>

 [Part 1: Numerical Linear Algebra](#label0)<br>
&emsp; [Chapter 1: A Review of Linear Algebra](#label1)<br>
&emsp; [Chapter 2: Direct Methods for Linear Systems](#label2)<br>
&emsp; [Chapter 3: Inconsistent Systems](#label3)<br>
&emsp; [Chapter 4: Computing Eigenvalues](#label4)<br>
&emsp; [Chapter 6: Fast Fourier Transform](#label5)<br>
 [Part 2: Numerical Methods for Analysis](#label6)<br>
&emsp; [Chapter 7: Preliminaries](#label7)<br>
&emsp; [Chapter 8: Solutions to Nonlinear Equations](#label8)<br>
&emsp; [Chapter 9: Interpolation](#label9)<br>
&emsp; [Chapter 10: Approximating Functions](#label10)<br>
&emsp; [Chapter 11: Differentiation and Integration](#label11)<br>
 [Part 3: Numerical Differential Equations](#label12)<br>
&emsp; [Chapter 12: Ordinary Differential Equations](#label13)<br>
&emsp; [Chapter 13: Parabolic Equations](#label14)<br>
&emsp; [Chapter 16: Fourier Spectral Methods](#label15)<br>
 [Part 4: Solutions](#label16)<br>
&emsp; [Numerical Linear Algebra](#label17)<br>
&emsp; [Numerical Analysis](#label18)<br>
&emsp; [Numerical Differential Equations](#label19)<br>


<a name="label0"></a>
# Part 1: Numerical Linear Algebra
<a name="label1"></a>
## Chapter 1: A Review of Linear Algebra
**The Hilbert matrix.** The Hilbert matrix $\mathbf{H}$ is ill-conditioned even for relatively small dimensions. Taking $\mathbf{H}^{-1}\mathbf{H}$ should give us the identity matrix.Notice that Python gives explicit warnings that the matrices are ill conditioned.

In [ ]:
M = [la.solve(la.hilbert(n),la.hilbert(n)) for n in [10,15,20,25,50]]
fig, ax = plt.subplots(1, 5)
for i in range(len(M)): 
  ax[i].imshow(1-np.abs(M[i]),vmin=0,cmap="gray")
plt.tight_layout(); plt.show()

<a name="label2"></a>
## Chapter 2: Direct Methods for Linear Systems
**Gaussian elimination.** The following function implements a naïve Gaussian elimination algorithm for a matrix `A` and vector `b`. We'll verify the code using a random matrix-vector pair. Note that the function `gaussian_elimination(A,b)` overwites both `A` and `b`. Pass array copies of these objects `gaussian_elimination(A.copy(),b.copy())` if you wish to avoid overwriting them.

In [ ]:
def gaussian_elimination(A,b):
  n = len(A)
  for j in range(n):
    A[j+1:,j] /= A[j,j]
    A[j+1:,j+1:] -= np.outer(A[j+1:,j],A[j,j+1:])
  for i in range(1,n):
    b[i] = b[i] - A[i,:i]@b[:i]
  for i in reversed(range(n)):
    b[i] = ( b[i] - A[i,i+1:]@b[i+1:] )/A[i,i]
  return(b)

In [ ]:
A = np.random.rand(8,8); b = np.random.rand(8,1)
np.c_[la.solve(A,b),gaussian_elimination(A,b)]

**Simplex method.** <a name="simplex"></a>The following three functions (`get_pivot`, `row_reduce`, and `simplex`) implement a naïve simplex method. Let's use them to solve the LP problem "Find the maximum of the objective function $2x + y + z$ subject to the constraints $2x+ z  \leq 3$, $4x+y + 2z  \leq 2$, and $x+y \leq 1$" along with the dual LP problem. 

In [ ]:
def get_pivot(tableau):
  j = np.argmax(tableau[-1,:-1]>0)
  a, b = tableau[:-1,j], tableau[:-1,-1]
  k = np.argwhere(a > 0)
  i = k[np.argmin(b[k]/a[k])]
  return(i,j)

In [ ]:
def row_reduce(tableau):
  i,j = get_pivot(tableau)
  G = tableau[i,:]/tableau[i,j]
  tableau -= tableau[:,j].reshape(-1,1)*G
  tableau[i,:] = G

In [ ]:
from collections import namedtuple
def simplex(c,A,b):
  m,n = A.shape
  tableau = np.r_[np.c_[A,np.eye(m),b], \
    np.c_[c.T,np.zeros((1,m)),0]]
  while (any(tableau[-1,:n]>0)): row_reduce(tableau)
  p = np.argwhere(tableau[-1,:n] != 0) 
  x = np.zeros(n)
  for i in p.flatten(): 
    x[i] = np.dot(tableau[:,i],tableau[:,-1])
  z = -tableau[-1,-1]
  y = -tableau[-1,range(n,n+m)]
  solution = namedtuple("solution",["z","x","y"])       
  return(solution(z,x,y))

In [ ]:
A = np.array([[2,0,1],[4,1,2],[1,1,0]])
c = np.array([[2],[1],[1]]);  b = np.array([[3],[2],[1]])
solution = simplex(c,A,b)
print("outcome:", solution.z)
print("primal solution:", solution.x)
print("dual solution:", solution.y)

**Graph drawing.** The following code draws the dolphin networks of the Doubtful Sound. We'll use dolphin gray (\#828e84) to color the nodes.<a name="dolphins_graph"></a>

In [ ]:
import pandas as pd, networkx as nx
df = pd.read_csv(bucket+'dolphins.csv', header=None)
G = nx.from_pandas_edgelist(df,0,1)
nx.draw(G,nx.spectral_layout(G),alpha=0.5,node_color='#828e84')
plt.axis('equal'); plt.show()

In [ ]:
nx.draw(G,nx.spring_layout(G),alpha=0.5,node_color='#828e84')
plt.axis('equal'); plt.show()

In [ ]:
nx.draw(G,nx.circular_layout(G),alpha=0.5,node_color='#828e84')
plt.axis('equal'); plt.show()

**Revised simplex method.** 

In [ ]:
from collections import namedtuple
def revised_simplex(c,A,b):
  c, A, b = c.astype(float), A.astype(float), b.astype(float)
  m, n = A.shape
  def xi(i): z=sp.lil_matrix((m,1)); z[i] = 1; return z.tocsr()
  N =  np.arange(n); B = np.arange(n,n+m)
  A = sp.hstack([sp.csr_matrix(A),sp.identity(m)],format="csr")
  ABinv = sp.identity(m).tocsr()
  b = sp.csr_matrix(b)
  c = sp.vstack([sp.csr_matrix(c),sp.csr_matrix((m,1))])
  while True:
    J = np.argwhere( (c[N].T-(c[B].T @ ABinv) @ A[:,N]) > 0)
    if len(J)==0: break
    j = J[0,1]   
    q = ABinv @ A[:,N[j]]
    k = np.argwhere(q>0)[:,0]
    i = k[ np.argmin( ABinv[k,:] @ b/q[k] ) ]
    B[i], N[j] = N[j], B[i]
    ABinv -=  ((q - xi(i))/q[i][0,0]) @ ABinv[i,:]
  i = np.argwhere(B<n)
  x = np.zeros(n)
  for k in i.flatten(): x[B[k]] = (ABinv[k,:] @ b)[0,0]
  y=(c[B].T@ABinv).toarray().flatten()
  solution = namedtuple("solution",["z","x","y"])  
  return solution(z=x@c[:n],x=x,y=y)

<a name="label3"></a>
## Chapter 3: Inconsistent Systems
**Zipf's law.**  Let's use ordinary least squares to find Zipf's law coefficients for canon of Sherlock Holmes.

In [ ]:
import pandas as pd
data = pd.read_csv(bucket+'sherlock.csv', sep='\t', header=None)
T = np.array(data[1])
n = len(T)
A = np.c_[np.ones((n,1)),np.log(np.arange(1,n+1)[:, np.newaxis])]
B = np.log(T)[:, np.newaxis]
c = la.lstsq(A,B)[0]
print('ordinary least squares:\n'+str(c))

**Constrained least squares.** The constrained least squares problem of solving $\mathbf{Ax} = \mathbf{b}$ with the constraint condition $\mathbf{Cx}=\mathbf{d}$:

In [ ]:
def constrained_lstsq(A,b,C,d):
  x = la.solve(np.r_[np.c_[A.T@A,C.T], 
      np.c_[C,np.zeros((C.shape[0],C.shape[0]))]], np.r_[A.T@b,d] )
  return(x[:A.shape[1]])

**Total least squares.**  The function `tls` solves the total least squares problem.

In [ ]:
def tls(A,B):
  n = A.shape[1]
  _,_,V = la.svd(np.c_[A,B])
  return(-la.solve(V[n:,n:],V[n:,:n]).T)

In [ ]:
A = np.array([[2,4],[1,-1],[3,1],[4,-8]]); b = np.array([[1,1,4,1]]).T
x_ols = la.lstsq(A,b)[0]
x_tls = tls(A,b)
print("ordinary least squares:", x_ols.T)
print("total least squares:", x_tls.T)

**Image compression** Let's use singular value decomposition to compress an image. We'll choose a nominal rank `k = 20` for demonstration. We'll use the Frobenius norm to compute the total pixelwise error in the compressed image. Then, we'll plot out all the singular values for comparison.

In [ ]:
import urllib
f = urllib.request.urlopen(bucket+'red-fox.jpg')
image = plt.imread(f,format='jpg')
A = np.dot(image[...,:3], [0.2989, 0.5870, 0.1140])
U,sigma,V = la.svd(A)

In [ ]:
k = 20
Ak = U[:,:k] @ np.diag(sigma[:k]) @ V[:k,:]
la.norm(A-Ak,'fro') - la.norm(sigma[k:])

In [ ]:
plt.imshow(Ak, cmap=plt.get_cmap('gray')); plt.show()
r = np.sum(A.shape)/np.prod(A.shape)*range(1,min(A.shape)+1)
error = 1 - np.sqrt(np.cumsum(sigma**2))/la.norm(sigma)
plt.semilogx(r,error,'.-'); plt.show()

**Non-negative matrix factorization (NMF).** A naive implementation of non-negative matrix factorization using multiplicative updates (without a stopping criterion):

In [ ]:
def nmf(X,p=6):
  W = np.random.rand(X.shape[0],p)
  H = np.random.rand(p,X.shape[1])
  for i in range(50):
    W = W*(X@H.T)/(W@(H@H.T) + (W==0))
    H = H*(W.T@X)/((W.T@W)@H + (H==0))
  return (W,H)

<a name="label4"></a>
## Chapter 4: Computing Eigenvalues
**Eigenvalue condition number.** The function `condeig` computes the eigenvalue condition number. Let's use it on a small random matrix.

In [ ]:
def condeig(A):
  w, vl, vr = la.eig(A, left=True, right=True)
  c = 1/np.sum(vl*vr,axis=0)
  return(c, vr, w)
A = np.random.rand(4,4)
condeig(A)

**PageRank.** The following minimal code computes the PageRank of the very  small graph by using the power method over 9 iterations <img src="https://raw.githubusercontent.com/nmfsc/data/master/internet_graph.svg" alt="internet graph" title="internet graph" />

In [ ]:
H = np.array([[0,0,0,0,1],[1,0,0,0,0], \
      [1,0,0,0,1],[1,0,1,0,0],[0,0,1,1,0]])
v = ~np.any(H,0) 
H = H/(np.sum(H,0)+v)
n = len(H) 
d = 0.85;
x = np.ones((n,1))/n
for i in range(9):
  x = d*(H@x) + d/n*(v@x)  + (1-d)/n
x

<a name="label5"></a>
## Chapter 6: Fast Fourier Transform
**Radix-2 FFT.** This chapter introduces several naive functions. The radix-2 FFT algorithm is written as a recursive function `fftx2` and the inverse FFT is written as `ifftx2`.<a name="radix2fft"></a>

In [ ]:
def fftx2(c):
  n = len(c)
  omega = np.exp(-2j*np.pi/n); 
  if np.mod(n,2) == 0:
    k = np.arange(n/2)
    u = fftx2(c[:-1:2])
    v = (omega**k)*fftx2(c[1::2])
    return( np.concatenate((u+v, u-v)) )
  else:
    k = np.arange(n)[:,None]
    F = omega**(k*k.T);
    return( (F @ c))

In [ ]:
def ifftx2(y): return(np.conj(fftx2(np.conj(y)))/len(y))

**Fast Toeplitz multiplication.** The function `fasttoeplitz` computes the Toeplitz multiplication by padding out a Toeplitz matrix with zeros to make it circulant.

In [ ]:
def fasttoeplitz(c,r,x):
  n = len(x)
  m = (1<<(n-1).bit_length())-n
  x1 = np.concatenate((np.pad(c,(0,m)),r[:1:-1]))
  x2 = np.pad(x,(0,m+n-1))
  return(ifftx2(fftx2(x1)*fftx2(x2))[:n])

**Bluestein algorithm.** The following function implements  the Bluestein algorithm using fast Toeplitz multiplication.

In [ ]:
def bluestein(x):
  n = len(x)
  w = np.exp((1j*np.pi/n)*(np.arange(n)**2))
  return(w*fasttoeplitz(conj(w),conj(w),w*x))

**Fast Poisson solver.** The following set of functions solves the Poisson equation using a naive fast Poisson solver.

<a name="label6"></a>
# Part 2: Numerical Methods for Analysis
<a name="label7"></a>
## Chapter 7: Preliminaries
Let's start with a function that returns a double-precision floating-point representation as a string of bits.

In [ ]:
def float_to_bin(x):
  if x == 0: return "0" * 64
  w, sign = (float.hex(x),0) if x > 0 else (float.hex(x)[1:],1)
  mantissa, exp = int(w[4:17], 16), int(w[18:])
  return "{}{:011b}{:052b}".format(sign, exp + 1023, mantissa)
float_to_bin(np.pi)

**Fast inverse square root.**  The following function implements the circa 1999 Q_rsqrt algorithm to approximate the reciprocal square root of a number.

In [ ]:
def Q_rsqrt(x):
  x = np.float32(x)
  i = x.view(int)
  i = 0x5f3759df - i//2 
  y = i.view(np.float32)  
  return y * (1.5 - (0.5 * x * y * y))

In [ ]:
Q_rsqrt(0.01)

**Rump's catastrophic cancellation.**  The answer should be `-0.827396...` What does Python come up with?

In [ ]:
a = 77617; b = 33096
333.75*b**6+a**2*(11*a**2*b**2-b**6-121*b**4-2)+5.5*b**8+a/(2*b)

`NaN` can be used to lift "pen off paper" when plotting a series of connected points.

In [ ]:
plt.plot(np.array([1,2,2,2,3]),np.array([1,2,np.nan,1,2]));plt.show()

<a name="label8"></a>
## Chapter 8: Solutions to Nonlinear Equations
We start with simple implementation of the bisection method.

In [ ]:
def bisection(f,a,b,tolerance):
  while abs(b-a)>tolerance:
    c = (a+b)/2
    if np.sign(f(c))==np.sign(f(a)): a = c 
    else: b = c
  return((a+b)/2)

In [ ]:
bisection(lambda x: np.sin(x),2,4,1e-14)

**The Mandelbrot set**. The following function takes the array `bb` for the lower-left and upper-right corners of the bounding box; `xpix` for the number of horizontal pixels; `n` for the maximum number of iterations; and `s` for the starting value $z^{(0)}$, which for the Mandelbrot set is 0. The function returns a two-dimensional array `M` that counts the number of iterations `k` to escape $\{z\in\C \mid |z^{(k)}|>2\}$. We'll then use this function to generate an image.

In [ ]:
def mandelbrot(bb,xpix,n,s):
  ypix = int(np.round(xpix*(bb[3]-bb[1])/(bb[2]-bb[0])))
  M = np.zeros((ypix,xpix))
  z = s*np.ones((ypix,xpix),dtype=complex)
  c = np.linspace(bb[0],bb[2],xpix).reshape(1,-1) + \
      1j*np.linspace(bb[3],bb[1],ypix).reshape(-1,1)
  for k in range(n):
    mask = np.abs(z)<2
    M[mask] += 1
    z[mask] = z[mask]**2 + c[mask]
  return(M)

In [ ]:
import matplotlib.image as mpimg
M = mandelbrot([-0.1710,1.0228,-0.1494,1.0443],800,200,0)
mpimg.imsave('mandelbrot.png', -M, cmap='magma')
from PIL import Image
Image.open('mandelbrot.png')

Imaging the Julia set uses almost identical code. The Mandelbrot set lives in the $c$-domain with a given value $z^{(0)}=0$, and the Julia set lives in the $z$-domain with a given value $c$. So the code for the Julia set requires only swapping the initializations for variables `z` and `c` in the code.

In [ ]:
def julia(bb,xpix,n,s):
  ypix = int(np.round(xpix*(bb[3]-bb[1])/(bb[2]-bb[0])))
  M = np.zeros((ypix,xpix))
  z = s*np.ones((ypix,xpix),dtype=complex)
  c = np.linspace(bb[0],bb[2],xpix).reshape(1,-1) + \
      1j*np.linspace(bb[3],bb[1],ypix).reshape(-1,1)
  c,z = z,c
  for k in range(n):
    mask = np.abs(z)<2
    M[mask] += 1
    z[mask] = z[mask]**2 + c[mask]
  return(M)

In [ ]:
J = julia([-2,-1,2,1],800,100,-1+0.3j)
mpimg.imsave('julia.png', -J, cmap='magma')
Image.open('julia.png')

**Homotopy continuation**. The following snippet of code finds a root of $$x^3-3xy^2-1 =0$$
$$y^3-3x^2y = 0$$ with an initial guess $(x,y) = (1,1)$ using homotopy continuation: 

In [ ]:
from scipy.integrate import solve_ivp
def f(x): return(np.array([x[0]**3-3*x[0]*x[1]**2-1, 
  x[1]**3-3*x[0]**2*x[1]]))
def df(t,x,p): 
  A = np.array([[3*x[0]**2-3*x[1]**2,-6*x[0]*x[1]],
      [-6*x[0]*x[1],3*x[1]**2-3*x[0]**2]])
  return(la.solve(-A,p))
x0 = np.array([1,1])
sol = solve_ivp(df,[0,1],x0,args=(f(x0),))
sol.y[:,-1]

<a name="label9"></a>
## Chapter 9: Interpolation
 **Splines.** The function `spline_natural` computes the coefficients `m` of a cubic spline with natural boundary conditions through the nodes given by the arrays `x` and `y`. The function `evaluate_spline` returns a set of `n` points along the spline. The final code snippet tests these functions with several randomly selected points.<a name="spline_natural"></a>

In [ ]:
def spline_natural(x,y):
  h = np.diff(x)
  gamma = 6*np.diff(np.diff(y)/h)
  C = [h[:-1],2*(h[:-1]+h[1:])]
  m = np.pad(la.solveh_banded(C,gamma),(1, 1))
  return(m)

In [ ]:
def evaluate_spline(x,y,m,n):
  h = np.diff(x)
  B = y[:-1] - m[:-1]*h**2/6
  A = np.diff(y)/h-h/6*np.diff(m)
  X = np.linspace(np.min(x),np.max(x),n+1)    
  i = np.array([np.argmin(i>=x)-1 for i in X])
  i[-1] = len(x)-2
  Y = (m[i]*(x[i+1]-X)**3 + m[i+1]*(X-x[i])**3)/(6*h[i]) \
      + A[i]*(X-x[i]) + B[i]
  return(X,Y)

In [ ]:
x = np.linspace(0,1,8); y = np.random.rand(8)
m = spline_natural(x,y)
X,Y = evaluate_spline(x,y,m,100)
plt.plot(x,y,'.',X,Y); plt.show()

**Bézier cirves.** The following function builds a Bernstein matrix. We'll then test the function on a set of points to create a cubic Bézier curve with a set of four randomly selected control points. 

In [ ]:
def bernstein(n,t): 
  from scipy.special import comb
  k = np.arange(n+1)[None,:]
  return(comb(n,k)*t**k*(1-t)**(n-k))

In [ ]:
n = 3
t = np.linspace(0,1,20)[:,None]
p = np.random.rand(n+1,2)
z = bernstein(n,t)@p
plt.plot(p[:,0],p[:,1],'.-',alpha=0.3);
plt.plot(z[:,0],z[:,1]); plt.show()

<a name="label10"></a>
## Chapter 10: Approximating Functions
 **Legendre polynomials** We can evaluate a Legendre polynomial of order $n$ using Bonnet's recursion formula.

In [ ]:
def legendre(x,n):
  if n==0: 
    return(np.ones_like(x))
  elif n==1:
    return(x)
  else:
    return(x*legendre(x,n-1)-1/(4-1/(n-1)**2)*legendre(x,n-2))

In [ ]:
x = np.linspace(-1,1,100)
for n in range(5): plt.plot(x,legendre(x,n))
plt.show()

**Chebyshev polynomials** We'll construct a Chebyshev differentiation matrix and use the matrix to solve a few simple problems.

In [ ]:
def chebdiff(n):
  x = -np.cos(np.linspace(0,np.pi,n))[:,None]
  c = np.outer(np.r_[2,np.ones(n-2),2],(-1)**np.arange(n))
  D = c/c.T/(x - x.T + np.eye(n))
  return D - np.diag(np.sum(D,axis=1)), x

In [ ]:
n = 15
D,x = chebdiff(n)
u = np.exp(-4*x**2);
plt.plot(x,D@u,'.-')
t = np.linspace(-1,1,200);
plt.plot(t,-8*t*np.exp(-4*t**2)); plt.show()

In [ ]:
D[0,:] = np.zeros((1,n)); D[0,0] = 1; u[0] = 2
plt.plot(x,la.solve(D,u),'.-'); plt.show()

In [ ]:
n = 15; k2 = 256
D,x = chebdiff(n)
L = D@D - k2*np.diag(x.flatten())
L[[0,-1],:] = 0; L[0,0] = 1; L[-1,-1] = 1 
y = la.solve(L,np.r_[2,[0]*(n-2),1].T)
plt.plot(x,y,'o');

In [ ]:
from scipy import special
k32 = np.cbrt(k2) 
ai,_,bi,_ = special.airy([-k32,k32])
a = la.solve(np.c_[ai,bi],np.c_[2,1].T)
def sol(x): 
  ai,_,bi,_ = special.airy(k32*x)
  return a[0]*ai + a[1]*bi
plt.plot(t,sol(t)); plt.show()

In [ ]:
N = range(6,62,2); e = []
for n in N: 
  D,x = chebdiff(n);
  L = D@D - k2*np.diag(x.flatten())
  L[[0,-1],:] = 0; L[0,0] = 1; L[-1,-1] = 1 
  y = la.solve(L,np.r_[2,[0]*(n-2),1][:,None])
  e.append(la.norm(y - sol(x),np.inf))
plt.semilogy(N,e,'.-');plt.show()

**Wavelets**. The function `scaling` returns the scaling function (father wavelet). We can use it to generate the wavelet function (mother wavelet). As an example, we will plot the Daubechies $D_4$ with $c_k = (1+\sqrt{3},3+\sqrt{3},3-\sqrt{3},1-\sqrt{3}])/4$ and $\phi(k) = (0,1+\sqrt{3},1-\sqrt{3},0)/2$.

In [ ]:
def scaling(c,z,n):
  m = len(c); L = 2**n
  phi = np.zeros(2*m*L) 
  phi[0:m*L:L] = z
  for j in range(n):
    for i in range(m*2**j):
      x = (2*i+1)*2**(n-1-j)            
      phi[x] = sum([c[k]*phi[(2*x-k*L)%(2*m*L)] for k in range(m)])
  return(np.arange((m-1)*L)/L,phi[:(m-1)*L])

In [ ]:
sqrt3 = np.sqrt(3)
c = np.array([1+sqrt3,3+sqrt3,3-sqrt3,1-sqrt3])/4
z = np.array([0,1+sqrt3,1-sqrt3,0])/2
x,phi = scaling(c,z,8)
plt.plot(x,phi); plt.show()

In [ ]:
psi = np.zeros_like(phi); n = len(c)-1; L = len(phi)//(2*n)
for k in range(n):
  psi[k*L:(k+n)*L] += (-1)**k*c[n-k]*phi[::2]

**DWT image compression** The following code explores using a discrete wavelet transform along with filtering as means of image compression.

In [ ]:
import pywt, urllib    
def rgb2gray(rgb): return np.dot(rgb[...,:3], [0.2989,0.5870,0.1140])
def adjustlevels(x): return 1-np.clip(np.sqrt(np.abs(x)),0,1)
f = urllib.request.urlopen(bucket+'laura_square.png')
img = rgb2gray(plt.imread(f,format='png'))
A = pywt.wavedec2(img,'haar')
c, slices = pywt.coeffs_to_array(A)
plt.imshow(adjustlevels(c), cmap='gray'); plt.axis('off'); plt.show()

In [ ]:
level = 0.5
c = pywt.threshold(c,level)
c = pywt.array_to_coeffs(c,slices,output_format='wavedec2')
B = pywt.waverec2(c,'haar')
plt.imshow(B, cmap='gray'); plt.axis('off'); plt.show()

**Nonlinear least squares approximation**. The function `gauss_newton` solves a nonlinear least squares problem where the Jacobian is approximated numerically using function `jacobian`. The solver is then used to find the parameters for a model logistic regression problem.<a name="jacobian"></a>

In [ ]:
def gauss_newton(x,y,c,f):
  r = y - f(c,x)
  for j in range(50):
    c += la.lstsq(jacobian(f,c,x),r)[0]
    r, r0 = y-f(c,x), r
    if la.norm(r-r0) < 1e-8: return(c)
  print('Gauss-Newton did not converge.')

In [ ]:
def jacobian(f,c,x):
  J = np.empty([len(x), len(c)])
  n = np.arange(len(c))
  for i in n:
    J[:,i] = np.imag(f(c+1e-8j*(i==n),x))/1e-8
  return(J)

In [ ]:
x = np.r_[np.random.normal(0,1,10),np.random.normal(0,1,10)+2]
y = np.r_[np.zeros(10),np.ones(10)]
def f(c,x): return(1/(np.exp(-c[0]*x+c[1])+1))
c = gauss_newton(x,y,[1,2],f)
X = np.linspace(-4,6,100)
plt.plot(x,y,'.',X,f(c,X)); plt.show()

**Neural Networks**. Let's use a neural network to find the a function that approximates semi-circular data.

In [ ]:
N = 100; θ = np.linspace(0,np.pi,N)
x = np.cos(θ); x = np.c_[np.ones(N),x].T
y = np.sin(θ) + 0.05*np.random.randn(1,N)

In [ ]:
n = 20; W1 = np.random.rand(n,2); W2 = np.random.randn(1,n)
def ϕ(x): return np.maximum(x,0)
def dϕ(x): return (x>0)
α = 1e-3

In [ ]:
for epoch in range(10000):
  ŷ = W2 @ ϕ(W1@x)
  dLdy = 2*(ŷ-y)
  dLdW1 = dϕ(W1@x)* (W2.T@ dLdy) @ x.T   
  dLdW2 = dLdy @ ϕ(W1@x).T
  W1 -= 0.1 * α * dLdW1 
  W2 -= α * dLdW2

In [ ]:
plt.scatter(x[1,:],y,color='#ff000050')
x̂ = np.linspace(-1.2,1.2,200); x̂ = np.c_[np.ones_like(x̂),x̂].T
ŷ = W2 @ ϕ(W1@x̂)
plt.plot(x̂[1,:],ŷ[0],color='#000000')
plt.show()

In [ ]:
N = 100; θ = np.linspace(0,np.pi,N)
x = np.cos(θ); x = np.c_[np.ones(N),x].T
y = np.sin(θ) + 0.05*np.random.randn(1,N)
n = 20; W1 = np.random.rand(n,2); W2 = np.random.randn(1,n)
def ϕ(x): return 1/(1+np.exp(-x))
def dϕ(x): return ϕ(x)*(1-ϕ(x))
α = 1e-1
for epoch in range(10000):
  ŷ = W2 @ ϕ(W1@x)
  L = la.norm(ŷ-y)
  dLdy = 2*(ŷ-y)/L
  dLdW1 = dϕ(W1@x)* (W2.T@ dLdy) @ x.T   
  dLdW2 = dLdy @ ϕ(W1@x).T
  W1 -= 0.1*α * dLdW1 
  W2 -= α * dLdW2 
plt.scatter(x[1,:],y,color='#ff000050')
x̂ = np.linspace(-1.2,1.2,200); x̂ = np.c_[np.ones_like(x̂),x̂].T
ŷ = W2 @ ϕ(W1@x̂)
plt.plot(x̂[1,:],ŷ[0],color='#000000');plt.ylim(-0.25,1.25)
plt.show()

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
n = 20; N = 100; 
θ = tf.linspace(0.0,np.pi,N)
x = tf.math.cos(θ); y = tf.math.sin(θ) + 0.05*tf.random.normal([N])

In [ ]:
model = keras.Sequential(
  [
    layers.Dense(n, input_dim=1, activation='relu'),
    layers.Dense(1)
  ]
)
model.compile(loss='mean_squared_error', optimizer='SGD')
model.fit(x,y,epochs=2000,verbose=0)
ŷ = model.predict(x)
plt.plot(x,ŷ,color='#000000'); plt.scatter(x,y,color='#ff000050')

In [ ]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.1, momentum=0.9)

<a name="label11"></a>
## Chapter 11: Differentiation and Integration
 Let's compute the coefficients to the third-order approximation to $f'(x)$ using nodes at $x-h$, $x$, $x+h$ and $x+2h$. We can use the function `rats` <a name="rats"></a> to rewrite the floating-point approximation for the coefficients given by `C[1,:]` as fractions

In [ ]:
d = np.array([-1,0,1,2])[:,None]
n = len(d)
V = d**np.arange(n) / [np.math.factorial(i) for i in range(n)]
C = la.inv(V)

In [ ]:
from fractions import Fraction
def rats(x): return str(Fraction(x).limit_denominator())
[rats(x) for x in C[1,:]]

**Richardson extrapolation**. $D(\phi(x))$ of a finite difference operator $\phi(x)$.<a name="richardson_extrapolation"></a>

In [ ]:
def richardson(f,x,m,n):
  if n==0: return(phi(f,x,2**m)) 
  return (4**n*richardson(f,x,m,n-1)-richardson(f,x,m-1,n-1))/(4**n-1)

In [ ]:
def phi(f,x,n): return((f(x+1/n) - f(x-1/n))/(2/n))

In [ ]:
richardson(lambda x: np.sin(x),0,4,4)

**Automatic differentiation**. <a name="dualclass"></a> We can build a minimal working example of forward accumulation automatic differentiation by defining a class and overloading the base operators.  We'll verify the code using the funtion $x + \sin x$. 

In [ ]:
class Dual:
  def __init__(self, value, deriv=1):
    self.value = value
    self.deriv = deriv
  def __add__(u, v):
    return Dual(value(u) + value(v), deriv(u) + deriv(v))
  __radd__ = __add__
  def __sub__(u, v):
    return Dual(value(u) - value(v), deriv(u) - deriv(v))
  __rsub__ = __sub__
  def __mul__(u, v):
    return Dual(value(u)*value(v), 
        value(v)*deriv(u) + value(u)*deriv(v))
  __rmul__ = __mul__
  def sin(u):
    return Dual(sin(value(u)),cos(value(u))*deriv(u))

In [ ]:
def value(x): 
  return(x.value if isinstance(x, Dual) else x)
def deriv(x): 
  return(x.deriv if isinstance(x, Dual) else 0)
def sin(x): return np.sin(x) 
def cos(x): return np.cos(x) 
def auto_diff(f,x): 
    return f(Dual(x)).deriv

In [ ]:
auto_diff(lambda x: x + sin(x),0)

Now, let's apply the code above to compute the Jacobian of the system $$y_1 = x_1x_2 + \sin x_2$$$$y_2 = x_1x_2 - \sin x_2$$ evaluated at $(x_1,x_2) = (2,\pi)$.

In [ ]:
x1 = Dual(2,np.array([1,0]))
x2 = Dual(np.pi,np.array([0,1]))
y1 = x1*x2 + sin(x2)
y2 = x1*x2 - sin(x2)
[y1.value,y2.value,y1.deriv,y2.deriv]

**Romberg method**. We can use the following trapezoidal quadrature  to make a Romberg method using Richardson extrapolation. We first define the function `trapezoidal` for composite trapezoidal quadrature. By redefine `phi` to equal `trapezoidal` we can simply apply the function `D` that we used to define Richardson extrapolation. We'll verify the code by integrating $\sin x$ from $0$ to $\pi/2$.

In [ ]:
def trapezoidal(f,x,n): 
  F = f(np.linspace(x[0],x[1],n+1))
  return((F[0]/2 + sum(F[1:-1]) + F[-1]/2)*(x[1]-x[0])/n)
def phi(f,x,n): return trapezoidal(f,x,n)
richardson(lambda x: np.sin(x),[0,np.pi/2],4,4)

**Composite trapezoidal method**. Let's examine the convergence rate for the composite trapezoidal rule applied to the function $f(x) = x + (x - x^2)^p$ over the interval $[0,2]$ with $p = 1,2,\dots,7$. We can do this by finding the loglog slope of the error as a function of subintervals $n$. We find that the error of the trapezoidal rule is $O(n^2)$ when $p=1$, $O(n^4)$ when $p$ is 2 or 3, $O(n^6)$ when $p$ is 4 or 5, and so on.

In [ ]:
n = np.logspace(1,2,num=10).astype(int)
error = np.zeros((10,7))
def f(x,p): return(x + x**p*(2-x)**p)
for p in range(1,8):
  S = trapezoidal(lambda x: f(x,p),(0,2),10**6)
  for i in range(len(n)):
    Sn = trapezoidal(lambda x: f(x,p),(0,2),n[i])
    error[i,p-1] =  abs(Sn - S)/S
np.log(error)  
A = np.c_[np.log(n),np.ones_like(n)]
x = np.log(error)
s = np.linalg.lstsq(A,x,rcond=None)[0][0]
info = ['{0}: slope={1:0.1f}'.format(k+1,s[k]) for k in range(7)]
lines = plt.loglog(n,error)
plt.legend(lines, info); plt.show()

**Clenshaw–Curtis quadrature**. applies the trapezoidal rule to a discrete cosine transform (type-1) as a means of numerically evaluating the integral $\int_{-1}^{1} f(x) \,\mathrm{d}x.$  We'll test the integral on the function $f(x) = 8 \cos x^2$, with an integral of approximately 0.566566

In [ ]:
def clenshaw_curtis(f,n):
  x = np.cos(np.pi*np.arange(n+1)/n)
  w = np.zeros(n+1); w[0:n+1:2] = 2/(1-np.arange(0,n+1,2)**2)
  return(1/n * np.dot(dctI(f(x)), w))

In [ ]:
from scipy.fft import dct
def dctI(f):
  g = dct(f,type=1)
  return(np.r_[g[0]/2, g[1:-1], g[-1]/2])

In [ ]:
clenshaw_curtis(lambda x: np.cos(8*x**2),20)

A mathematical comment: we could have also  defined a type-1 DCT explicitly in terms of its underlying FFTs if we wanted to crack the black box open just a wee bit more.

In [ ]:
from scipy.fft import fft
def dctI(f):
  n = len(f)
  g = np.real(fft(np.r_[f, f[n-2:0:-1]]))
  return(np.r_[g[0]/2, g[1:n-1], g[n-1]]/2)

**Gauss–Legendre quadrature.** We first compute the Legendre weights and nodes and then apply Gauss–Legendre quadrature to compute $$\int_{-1}^{1} \cos x \cdot \mathrm{e}^{-x^2} \,\mathrm{d}x$$ using a nominal number of nodes $n=8$.  Alternatively, we can use the NumPy  function `leggauss` to compute the nodes and weights for Gauss–Legendre quadrature.

In [ ]:
def gauss_legendre(n):
  a = np.zeros(n)  
  b = np.arange(1,n)**2 / (4*np.arange(1,n)**2 - 1)
  scaling = 2
  nodes, v = la.eigh_tridiagonal(a, np.sqrt(b))
  return(nodes, scaling*v[0,:]**2)

In [ ]:
n = 8
def f(x): return(np.cos(x)*np.exp(-x**2))
nodes, weights = gauss_legendre(n)
np.dot(f(nodes), weights)

In [ ]:
nodes, weights = np.polynomial.legendre.leggauss(n)
np.dot(f(nodes), weights)

<a name="label12"></a>
# Part 3: Numerical Differential Equations
<a name="label13"></a>
## Chapter 12: Ordinary Differential Equations
 Let's plot the boundary of the region of absolute stability for BDF2: $$z = \frac{\frac{3}{2} r^2 - 2 r + \frac{1}{2}}{r^2}$$

In [ ]:
r = np.exp(2j*np.pi*np.linspace(0,1,100))
z = (3/2*r**2 - 2*r + 0.5)/r**2
plt.plot(z.real,z.imag); plt.axis('equal'); plt.show()

**Multistep coefficients**. The function `multistepcoeffs` determines the multistep coefficients for stencil given by `m` and `n`. The function `plotstability` uses these coefficients to plot boundary of the region of absolute stability. We'll test it on the Adams–Moulton method with input `m = [1]` and `n = [0 1 2]`.<a name="multistepcoeffs"></a>

In [ ]:
def multistepcoeffs(m,n):
  s = len(m) + len(n)
  A = (np.r_[m]+1.0)**np.c_[range(s)]
  B = np.c_[range(s)]*(np.r_[n]+1.0)**(np.c_[range(s)]-1)
  c = la.solve(-np.c_[A,B],np.ones((s,1))).flatten()
  return(np.r_[1,c[:len(m)]], c[len(m):] )

In [ ]:
def plotstability(a,b):
  r = np.exp(1j*np.linspace(0,2*np.pi,200))
  z = [np.dot(a,r**np.arange(len(a))) / \
       np.dot(b, r**np.arange(len(b))) for r in r]
  plt.plot(np.real(z),np.imag(z));
  plt.axis('equal'); plt.show()

In [ ]:
m = [1]; n = [0,1,2]
a,b = multistepcoeffs(m,n)
plotstability(a,b)

**Recipe for solving an ODE**. The general recipe for solving an ODE is to
1. Load the module
1. Set up the parameters
1. Choose the method
1. Solve the problem
1. Present the solution

Let's apply this recipe to solve the pendulum problem $u'' = \sin u$ with initial conditions $u(0) = \pi/9$ and $u'(0) = 0$ over $t\in[0,8\pi]$.

In [ ]:
import numpy as np; import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
def pndlm(t,u): return u[1],-np.sin(u[0])
u0 = [8*np.pi/9,0]; tspan = [0,8*np.pi]
mthd = 'RK23'
sltn = solve_ivp(pndlm,tspan,u0,method=mthd)
plt.plot(sltn.t,sltn.y[0,:],'.-')
plt.plot(sltn.t,sltn.y[1,:],'.-')
plt.legend(labels=['θ','ω']); plt.show()

The values of `sltn.t` are those determined and used for adaptive time stepping. Higher-order methods such as `DOP853` that use smoother interpolating polynomials produce rougher (though still accurate) plots than lower-order methods such as `RK23`:

In [ ]:
mthd = 'DOP853'
sltn = solve_ivp(pndlm,tspan,u0,method=mthd)
plt.plot(sltn.t,sltn.y[0,:],'.-')
plt.plot(sltn.t,sltn.y[1,:],'.-')
plt.legend(labels=['θ','ω']); plt.show()

We can request a continuous solution by setting `dense_output=True`.  In this case, `solve_ivp` returns an additional field `sol` that we can think of as a function of the independent variable:

In [ ]:
sltn = solve_ivp(pndlm,tspan,u0,method=mthd,dense_output=True)
t = np.linspace(tspan[0],tspan[1],200)
y = sltn.sol(t)
plt.plot(t,y[0],t,y[1])
plt.legend(labels=['θ','ω']); plt.show()

<a name="label14"></a>
## Chapter 13: Parabolic Equations
**Heat equation using the backward Euler method**. Let's solve the heat equation using the backward Euler method with initial conditions given by a rectangular function and absorbing boundary conditions.

In [ ]:
import timeit
start_time = timeit.default_timer()
dx = .01; dt = .01; L = 2; c = dt/dx**2; uL = 0; uR = 0;
x = np.arange(-L,L,dx); n = len(x) 
u = (abs(x)<1)
u[0] += 2*c*uL; u[n-1] += 2*c*uR; 
D = np.tile(np.array([[-c,1+2*c,-c]]).T,(1,n))
D[0,1] = 0; D[2,n-2] = 0
for i in range(20):
  u = la.solve_banded((1, 1), D, u)
elapsed_time = timeit.default_timer() - start_time
plt.plot(x,u);plt.show()
print("elapsed time = "+str(elapsed_time))

**Heat equation using the Crank–Nicolson method**. Let's solve the heat equation again using the Crank–Nicolson method with initial conditions given by a rectangular function. This time we'll use reflecting boundary conditions. Notice how the high-frequency information does not decay as it did when using the backward Euler method.

In [ ]:
dx = .01; dt = .01; L = 2; λ = dt/dx**2
x = np.arange(-L,L,dx); n = len(x) 
u = (abs(x)<1)
diagonals = [np.ones(n-1), -2*np.ones(n), np.ones(n-1)]
D = sp.diags(diagonals, [-1,0,1], format='csr')
D[0,1] *= 2; D[-1,-2] *= 2
A = 2*sp.identity(n) + λ*D 
B = 2*sp.identity(n) - λ*D 
for i in range(20):
  u = sp.linalg.spsolve(B,A@u)
plt.plot(x,u); plt.show()

**Porous medium equation**. We'll now solve the porous medium equation $u_t = (u^2u_x)_x$ using an adaptive-step BDF routine.

In [ ]:
from scipy.integrate import solve_ivp
n = 400; L = 2; x,dx = np.linspace(-L,L,n,retstep=True)
def m(u): return u**2
def Du(t,u): 
  return(np.r_[0,np.diff(m((u[:-1]+u[1:])/2)*np.diff(u))/dx**2,0])
u0 = (abs(x)<1)
sol = solve_ivp(Du, [0,2], u0, method='LSODA',\
  lband=1,uband=1,dense_output=True)

In [ ]:
from ipywidgets import interactive
def anim(t=0):
  plt.fill_between(x,sol.sol(t),color='#ff9999');
  plt.ylim(0,1);plt.show()
interactive(anim, t = (0,2,0.01))

<a name="label15"></a>
## Chapter 16: Fourier Spectral Methods
**Heat equation**. The formal solution to the heat equation is $$u(t,x) = \mathrm{F}^{-1}\left[  \mathrm{e}^{-\xi^2 t}  \mathrm{F} u(0,x) \right].$$

In [ ]:
from numpy.fft import fft,ifft, fftfreq
n = 256; ℓ = 4
k2 = (fftfreq(n,1/n)*(2*np.pi/L))**2
def u(t,u0): return np.real(ifft(np.exp(-k2*t)*fft(u0)))

In [ ]:
x = np.linspace(-L/2,L/2,n,endpoint=False)
u0 = (np.abs(x)<1)
plt.plot(x,u(0.1,u0)); plt.show()

**Incompressible Navier–Stokes equation**. Let's solve the heat equation using the backward Euler method with initial conditions given by a rectangular function and absorbing boundary conditions.

In [ ]:
from numpy.fft import fft2,ifft2,fftfreq
def cdiff(Q,step=1): return Q-np.roll(Q,step,0)
def flux(Q,c): return c*cdiff(Q,1) - \
  0.5*c*(1-c)*(cdiff(Q,1)+cdiff(Q,-1))
def H(u,v,ikx,iky): return -ikx*fft2(ifft2(u)**2) + \
  -iky*fft2(ifft2(u)*ifft2(v))

In [ ]:
L, n, dt, e = 2, 128, 0.001, 0.001; dx = L/n 
x = np.linspace(dx,L,n)[None,:]; y = x.T
q = 0.5*(1+np.tanh(10*(1-np.abs(L/2 - y)/(L/4))))
Q = np.tile(q, (1,n))
u = Q*(1+0.5*np.sin(L*np.pi*x))  
v = np.zeros_like(u) 
u,v = fft2(u),fft2(v)
us,vs = u,v
ikx = (1j*fftfreq(n)*n*(2*np.pi/L))[None,:]
iky = ikx.T
k2 = ikx**2+iky**2
Hx, Hy = H(u,v,ikx,iky), H(v,u,iky,ikx)  
M1 = 1/dt + (e/2)*k2
M2 = 1/dt - (e/2)*k2

In [ ]:
for i in range(1200):
  Q -= flux(Q,(dt/dx)*np.real(ifft2(v))) + \
    flux(Q.T,(dt/dx)*np.real(ifft2(u)).T).T
  Hxo, Hyo = Hx, Hy
  Hx, Hy = H(u,v,ikx,iky), H(v,u,iky,ikx)           
  us = u - us + (1.5*Hx - 0.5*Hxo + M1*u)/M2
  vs = v - vs + (1.5*Hy - 0.5*Hyo + M1*v)/M2
  phi = (ikx*us + iky*vs)/(k2+(k2==0)) 
  u, v =  us - ikx*phi, vs - iky*phi
plt.imshow(Q,'seismic'); plt.show()

<a name="label16"></a>
# Part 4: Solutions
<a name="label17"></a>
## Numerical Linear Algebra
**1.4. Invertibility of random (0,1) matrices.** The number of invertible $n\times n$ (0,1) matrices is known for $n$ up to 8. (See the [On-Line Encyclopedia of Integer Sequences](http://oeis.org/A055165).) We'll approximate the ratio of invertible matrices by checking the determinants of randomly drawn ones. 

In [ ]:
N = 10000; n = np.zeros(20)
def mat_01(d): return(np.random.choice((0,1),size=(d,d)))
for d in range(20):
  n[d] = sum([np.linalg.det(mat_01(d))!=0 for i in range(N)])
plt.plot(range(1,21),n/N,'.-'); plt.show()

**2.3. Naive algorithm for the determinant.** The determinant of  matrix $\mathbf{A}$ is given by the product of the elements along the diagonal of $\mathbf{U}$ multiplied by the parity of the permutation matrix $\mathbf{P}$ from the LU decomposition $\mathbf{PLU} = \mathbf{A}$.

In [ ]:
def det(A):
  P,L,U = la.lu(A)
  s = 1
  for i in range(len(P)):
    try:
      m = np.argwhere(P[i+1:,i]).item(0)+1
      P[[i,i+m],:] = P[[i+m,i],:] 
      s *= -1
    except:
      pass
  return(s * np.prod(np.diagonal(U)))

In [ ]:
A = np.random.rand(20,20)
det(A) - la.det(A)

**2.4. Reverse Cuthill–McKee algorithm.** The following function implements a naive reverse Cuthill–McKee algorithm  for symmetric matrices. We'll verify the algorithm by applying it to a sparse, random (0,1) matrix.

In [ ]:
def rcuthillmckee(A):
  r = np.argsort(np.bincount(A.nonzero()[0]))
  while r.size:
    q = np.atleast_1d(r[0])
    r = np.delete(r,0)
    while q.size:
      try: p = np.append(p,q[0])
      except: p = np.atleast_1d(q[0])
      k = sp.find(A[q[0],r])[1]
      q = np.append(q[1:],r[k])
      r = np.delete(r,k)
  return(np.flip(p))

In [ ]:
A = sp.random(1000,1000,0.001); A += A.T
p = rcuthillmckee(A)
fig, (ax1, ax2) = plt.subplots(1, 2)
ax1.spy(A,ms=1); ax2.spy(A[p[:,None],p],ms=1)
plt.show()

**2.5. Dolphins of Doubtful Sound.**  We'll reuse the code [above](#dolphins_graph) used to draw the original graph of the dolphins.

In [ ]:
import pandas as pd, networkx as nx
df = pd.read_csv(bucket+'dolphins.csv', header=None)
G = nx.from_pandas_edgelist(df,0,1)
nx.draw(G,nx.circular_layout(G),alpha=0.5,node_color='#828e84')
plt.axis('equal'); plt.show()

In [ ]:
A = nx.adjacency_matrix(G)
p = rcuthillmckee(A)
A = A[p[:,None],p]
G = nx.from_scipy_sparse_matrix(A)
nx.draw(G,nx.circular_layout(G),alpha=0.5,node_color='#828e84')
plt.axis('equal'); plt.show()

**2.6. Stigler diet problem.** Let's solve the Stigler diet problem. We'll use the function naïve [`simplex`](#simplex) defined above.

In [ ]:
def get_pivot(tableau):
  j = np.argmax(tableau[-1,:-1]>0)
  a, b = tableau[:-1,j], tableau[:-1,-1]
  k = np.argwhere(a > 0)
  i = k[np.argmin(b[k]/a[k])]
  return(i,j)

def row_reduce(tableau):
  i,j = get_pivot(tableau)
  G = tableau[i,:]/tableau[i,j]
  tableau -= tableau[:,j].reshape(-1,1)*G
  tableau[i,:] = G

from collections import namedtuple
def simplex(c,A,b):
  m,n = A.shape
  tableau = np.r_[np.c_[A,np.eye(m),b], \
    np.c_[c.T,np.zeros((1,m)),0]]
  while (any(tableau[-1,:n]>0)): row_reduce(tableau)
  p = np.argwhere(tableau[-1,:n] != 0) 
  x = np.zeros(n)
  for i in p.flatten(): 
    x[i] = np.dot(tableau[:,i],tableau[:,-1])
  z = -tableau[-1,-1]
  y = -tableau[-1,range(n,n+m)]
  solution = namedtuple("solution",["z","x","y"])       
  return(solution(z,x,y))

In [ ]:
import pandas as pd
diet = pd.read_csv(bucket+'diet.csv')
A = diet.values[1:,3:].T
b = diet.values[0,3:][:,None]
c = np.ones(((A.shape)[1],1))
food = diet.values[1:,0]
solution = simplex(b,A.T,c)
print("value: ", solution.z)
i = np.argwhere(solution.y!=0).flatten()
print("foods: ", food[i])

In practice, we can use `scipy.optimize.linprog`.

In [ ]:
from scipy import optimize 
solution = optimize.linprog(c,-A,-b,method='revised simplex')
solution.fun, food[np.where(solution.x>1e-12)]

**2.7. Six degrees of Kevin Bacon** Let's determine the shortest path between two actors along with their connecting movies. We'll first define helper functions `get_names`<a href="get_names"></a> and `get_adjacency_matrix`<a href="get_adjacency_matrix"></a>. Then we'll build an biadjacency matrix $\mathbf{B

In [ ]:
def get_names(filename):
  return np.genfromtxt(bucket+filename+'.txt',delimiter='\n',\
    dtype="str",encoding="utf8").tolist()
def get_adjacency_matrix(filename):
  i = np.genfromtxt(bucket+filename+'.csv',delimiter=',',dtype=int)
  return sp.csr_matrix((np.ones_like(i[:,0]), (i[:,0]-1,i[:,1]-1)))

In [ ]:
actors = get_names("actors")
movies = get_names("movies")
B = get_adjacency_matrix("actor-movie")
A = sp.bmat([[None,B.T],[B,None]],format='csr')
actormovie = np.r_[actors,movies]

In [ ]:
def findpath(A,a,b):
  p = -np.ones(A.shape[1],dtype=np.int64)
  q = [a]; p[a] = -9999; i = 0
  while i<len(q):
    k = sp.find(A[q[i],:])[1]
    k = k[p[k]==-1]
    q.extend(k)
    p[k] = q[i]; i += 1
    if any(k==b): return(backtrack(p,b))
  display("No path.")

In [ ]:
def backtrack(p,b):
  s = [b]; i = p[b]
  while i != -9999: s.append(i); i = p[i]
  return s[::-1]

In [ ]:
a = actors.index("Bruce Lee"); b = actors.index("Tommy Wiseau")
actormovie[findpath(A,a,b)].tolist()

In practice, we can use the `shortest_path` function from the scipy.sparse.csgraph library or networkx library

In [ ]:
a = actors.index("Bruce Lee"); b = actors.index("Tommy Wiseau")
_,p = sp.csgraph.shortest_path(A,indices=a,return_predecessors=True)
actormovie[backtrack(p,b)].tolist()

In [ ]:
import networkx as nx
G = nx.from_scipy_sparse_matrix(A)
a = actors.index("Emma Watson"); b = actors.index("Kevin Bacon")
i = nx.shortest_path(G,a,b)
actormovie[i].tolist()

**3.4. NIST Filippelli problem.** The Filippelli dataset was contrived by NIST to benchmark linear regression software. The Filippelli problem consists of fitting an 10th degree polynomial to the data set⁠—a rather ill-conditioned problem. We first need to download the data. Then we'll define three methods for solving the Vandermonde problem: the normal equation, QR decomposition, and the pseudoinverse.

In [ ]:
import pandas as pd
df = pd.read_csv(bucket+'filip.csv',header=None)
y,x = np.array(df[0]),np.array(df[1])
coef = pd.read_csv(bucket+'filip-coeffs.csv',header=None)
beta = np.array(coef[0])[None,:]

In [ ]:
def solve_filip(x,y):
  V = vandermonde(x,11)
  Q,R = la.qr(V,mode='economic') 
  c = np.zeros((3,11),float)
  c[0,:] = la.solve(V.T@V,V.T@y)
  c[1,:] = la.solve(R,Q.T@y)
  c[2,:] = la.pinv(V,1e-14)@y
  r = [la.norm(V@c[i,:].T-y) for i in range(3)]
  return(c,r)
def build_poly(c,x): 
  return vandermonde(x,len(c))@c
def vandermonde(x,n): 
  return np.vander(x,n,increasing=True)

Now, let's solve the problem and plot the results. Let's also list the coefficients from each method alongside the official NIST coefficients. What do you notice about the coefficients? What method performs the best?

In [ ]:
b,r = solve_filip(x,y)
X = np.linspace(min(x),max(x),200)
b = np.r_[b,beta]
plt.scatter(x,y,color='#0000ff40')
for i in range(4): 
  plt.plot(X,build_poly(b[i],X))
plt.ylim(0.7,0.95);plt.show()
coef.assign(β1=b[0], β2=b[1], β3=b[2])

What makes the Filippelli problem a difficult problem is that the condition number of the system is huge. We can reduce the condition number by first standardizing the data before using it⁠—i.e., subtracting the mean and dividing by the standard deviation. Look at the difference in condition numbers of the Vandermonde matrix before and after standardizing the data.

In [ ]:
def zscore(X,x): return((X - x.mean())/x.std())
k1 = np.linalg.cond(vandermonde(x,11))
k2 = np.linalg.cond(vandermonde(zscore(x,x),11))
print("Condition numbers of the Vandermonde matrix:")
print("{:e}".format(k1))
print("{:e}".format(k2))

In [ ]:
c,r = solve_filip(zscore(x,x),zscore(y,y))
plt.scatter(x,y,color='#0000ff40')
for i in range(3):
  Y = build_poly(c[i],zscore(X,x))*y.std() + y.mean()
  plt.plot(X, Y)
plt.show()
la.norm(c[0]-c[1]),la.norm(c[0]-c[2])

**3.5. Modeling daily temperatures** We'll use $u(t) = c_1 \sin(2\pi t) + c_2 \cos(2\pi t) + c_3$ to model the daily temperatures using data recorded in Washington, DC. between 1967 and 1971.

In [ ]:
import pandas as pd
df = pd.read_csv(bucket+'dailytemps.csv')
t = pd.to_datetime(df["date"]).values
day = (t - t[0])/np.timedelta64(365, 'D')
u = df["temperature"].values[:,None]
def tempsmodel(t): return np.c_[np.sin(2*np.pi*t),\
  np.cos(2*np.pi*t), np.ones_like(t)]
c = la.lstsq(tempsmodel(day),u)[0]
plt.plot(day,u,'o',color='#0000ff15');
plt.plot(day,tempsmodel(day)@c,'k'); plt.show()

**3.6. Image recognition** We'll practice  identifying handwritten digits using the EMNIST database.  The [NIST website](https://www.nist.gov/itl/products-and-services/emnist-dataset) provides a version formatted as a Matlab MAT-file. The full container from the NIST website is quite large (around 700MB) and we only need a smaller 20MB set, so I've put a copy of the just the emnist-nist file on GitHub at this [link](https://github.com/nmfsc/data/raw/master/emnist-mnist.mat).

In [ ]:
from keras.datasets import mnist
import scipy.sparse.linalg
(image_train,label_train),(image_test,label_test) = mnist.load_data()
image_train = np.reshape(image_train, (60000,-1));
V = np.zeros((12,784,10))
for i in range(10):
  D = sp.csr_matrix(image_train[label_train==i], dtype=float)
  U,S,V[:,:,i] = sp.linalg.svds(D,12)

In [ ]:
import matplotlib.pyplot as plt
pix = [V[i,:,3].reshape(28,28) for i in range(11,-1,-1)]
plt.imshow(np.hstack(pix), cmap="gray")
plt.axis('off'); plt.show()

In [ ]:
image_test = np.reshape(image_test, (10000,-1));
r = np.zeros((10,10000))
for i in range(10):
  q = V[:,:,i].T@(V[:,:,i] @ image_test.T) -  image_test.T
  r[i,:]  = np.sum(q**2,axis=0)
prediction = np.argmin(r,axis=0)
confusion = np.zeros((10,10)).astype(int)
for i in range(10): 
  confusion[i,:] = np.bincount(prediction[label_test==i],minlength=10)

In [ ]:
import pandas as pd
pd.DataFrame(confusion)

In [ ]:
import requests
from scipy import io
r = requests.get(bucket+"emnist-mnist.mat")
with open('emnist-mnist.mat', 'wb') as local_file:
    local_file.write(r.content)
data = io.loadmat('emnist-mnist.mat')
image_train = data['dataset'][0,0]['train'][0,0]['images']
label_train = data['dataset'][0,0]['train'][0,0]['labels'].flatten()
image_test = data['dataset'][0,0]['test'][0,0]['images']
label_test = data['dataset'][0,0]['test'][0,0]['labels'].flatten()

**3.8. Actor similarity model** We use SVD to find a lower-dimensional subspace relating actors and genres. Then we find the closest actors in that subspace using cosine similarity. We use the functions [`get_names`](#get_names) and  [`get_adjacency_matrix`](#get_adjacency_matrix) developed earlier.

In [ ]:
actors = get_names("actors")
genres = get_names("genres")
A = get_adjacency_matrix("movie-genre"); A /= A.sum(axis=0)
B = get_adjacency_matrix("actor-movie")

In [ ]:
U,S,Vt = sp.linalg.svds(A@B, 12)
Q = Vt/np.sqrt((Vt**2).sum(axis=0))

In [ ]:
q = Q[:,actors.index("Steve Martin")]                         
z = Q.T@q
r = np.argsort(-z)
[actors[i] for i in r[:10]]

Let's also see Steve Martin's genre signature.

In [ ]:
p = (U*S)@q
r = np.argsort(-p)
[(genres[i],p[i]/p.sum()) for i in r[:10]]

**3.9. Multilateration** We use ordinary least squares and total least squares to solve a multulateration problem.

In [ ]:
xyt = np.array([[3,3,12],[1,15,14],[10,2,13],[12,15,13],[0,11,12]])
reference = xyt[0,:];  xyt = xyt - reference
A = np.array([2,2,-2])*xyt
b = (xyt**2)@np.array([[1],[1],[-1]])
x_ols = la.lstsq(A,b)[0] + reference[:,None]
x_tls = tls(A,b) + reference[:,None]

**4.2. Girko's circular law** Let's plot out the eigenvalues of a few thousand normal random matrices of size $n$ to get a probability distribution in the complex plane.  What do you notice?

In [ ]:
n = 8
E = [la.eigvals(np.random.randn(n,n)) for i in range(2500)]
E = np.concatenate(E)
plt.plot(E.real, E.imag,'.',c='#0000ff10',mec='none') 
plt.axis('equal'); plt.show()

**4.4. Rayleigh quotient iteration** Let's define a function that finds an eigenvalue $\lambda$ and eigenvector $\mathbf{x}$ of a matrix. We'll pick a random initial guess for $\mathbf{x}$ unless one is given. We'll then verify the algorithm on a symmetric matrix. Rayleigh quotient iteration works on general classes of matrices, but often has difficulty converging when matrices get large or far from symmetric⁠—i.e., when eigenvectors get closer together.

In [ ]:
def rayleigh(A,x=[]):
  n = len(A)
  if x==[]: x = np.random.randn(n,1)
  while True:
    x = x/la.norm(x)
    rho = x.T @ A @ x
    M = A - rho*np.eye(n)
    if abs(la.det(M))<1e-10:
      return(rho,x)
    x = la.solve(M,x)

**4.5. Implicit QR method** We'll define a function that computes all the eigenvalues of a matrix using the implicit QR method. We'll then verify the algorithm on a matrix with known eigenvalues.

In [ ]:
def implicitqr(A):
  tolerance = 1e-12
  n = len(A)
  H = la.hessenberg(A)
  while True:
    if abs(H[n-1,n-2]) < tolerance:
      n -= 1
      if n<2: return(np.diag(H))
    Q,_ = la.qr([[H[0,0]-H[n-1,n-1]], [H[1,0]]])
    H[:2,:n] = Q @ H[:2,:n]
    H[:n,:2] = H[:n,:2] @ Q.T
    for i in range(1,n-1):
      Q,_ = la.qr([[H[i,i-1]], [H[i+1,i-1]]])
      H[i:i+2,:n] = Q @ H[i:i+2,:n]
      H[:n,i:i+2] = H[:n,i:i+2] @ Q.T

In [ ]:
n = 20; S = np.random.randn(n,n); 
D = np.diag(np.arange(1,n+1)); A = S@D@la.inv(S)
implicitqr(A)

**4.6. Randomized SVD** We define a method that implements randomized SVD. The idea is to start with a set of $k$ random vectors and perform a few steps of the simple QR method to generate a $k$-dimensional subspace that is closer to the space of dominant singular values. Then, we perform SVD on that subspace. We may not get the exact singular values, but it we just need a good guess and the matrix is huge, this method can be significantly faster than SVD or sparse SVD.

In [ ]:
def randomizedsvd(A,k):
  Z = np.random.rand(A.shape[1],k)
  Q,R = la.qr(A@Z, mode='economic')
  for i in range(4):
    Q,R = la.qr(A.T @ Q, mode='economic')
    Q,R = la.qr(A @ Q, mode='economic')
  W,S,V = la.svd(Q.T @ A,full_matrices=False)
  U = Q @ W
  return(U,S,V)

In [ ]:
import timeit
n = 1000; k = 10
A = np.random.randn(n,n)
print('Random SVD singular values and elapsed time:')
start_time = timeit.default_timer()
display(randomizedsvd(A,k)[1])
display(timeit.default_timer() - start_time)
print('Regular SVD singular values and elapsed time:')
start_time = timeit.default_timer()
display(la.svd(A)[1][:k])
display(timeit.default_timer() - start_time)

**4.8. Hollywood eigenvector centrality** We'll use eigenvector centrality to determine who's at the center of Hollywood. We use the functions [`get_names`](#get_names) and  [`get_adjacency_matrix`](#get_adjacency_matrix) developed earlier.

In [ ]:
actors = get_names("actors")
B = get_adjacency_matrix("actor-movie")
r,c = (B.T@B).nonzero()
M = sp.csr_matrix((np.ones(len(r)),(r,c)))
v = np.ones(M.shape[0])
for k in range(10):
  v = M@v; v /= np.linalg.norm(v)
r = np.argsort(-v)
[actors[i] for i in r[:10]]

**5.3. 3D Poisson equation** Let's compare the Jacobi, Gauss-Seidel, SOR, and conjugate gradient methods in solving the Poisson equation in the unit cube.

In [ ]:
n = 50; xi = np.arange(1,n+1)/(n+1); dx = 1/(n+1)
I = sp.identity(n)
D = sp.diags([1, -2, 1], [-1, 0, 1], shape=(n, n))
A = ( sp.kron(sp.kron(D,I),I) + sp.kron(I,sp.kron(D,I)) +
  sp.kron(I,sp.kron(I,D)) )/dx**2
f = np.array([(x-x**2)*(y-y**2) + (x-x**2)*(z-z**2)+(y-y**2)*(z-z**2) 
    for x in xi for y in xi for z in xi])
ue = np.array([(x-x**2)*(y-y**2)*(z-z**2)/2 
    for x in xi for y in xi for z in xi])

In [ ]:
def stationary(A,b,w=0,n=400):
  e = []; u = np.zeros_like(b)
  P = sp.diags(A.diagonal(),0) + w*sp.tril(A,-1)
  for i in range(n):
    u += sp.linalg.spsolve(P,b-A@u,'NATURAL')
    e = np.append(e,la.norm(u - ue,1))
  return(e)

In [ ]:
def conjugategradient(A,b,n=400):
  e = []; u = np.zeros_like(b)
  r = b - A@u; p = np.copy(r)
  for i in range(n):
    Ap = A@p
    α = np.dot(r,p)/np.dot(Ap,p)
    u += α*p; r -= α*Ap
    β = np.dot(r,Ap)/np.dot(Ap,p)
    p = r - β*p
    e = np.append(e,la.norm(u - ue,1))
  return(e)

In [ ]:
e = np.zeros((400,4))
e[:,0] = stationary(A,-f,0)
e[:,1] = stationary(A,-f,1)
e[:,2] = stationary(A,-f,1.9)
e[:,3] = conjugategradient(A,-f)
plt.semilogy(e); 
plt.legend(["Jacobi","Gauss-Seidel","SOR","Conj. Grad."]); plt.show()

**6.1. Radix-3 FFT** The radix-3 FFT is similar to the [radix-2 FFT](#radix2fft). We'll verify that the code is correct by comparing it with a built-in FFT

In [ ]:
def fftx3(c):
  n = len(c)
  omega = np.exp(-2j*np.pi/n); 
  if np.mod(n,3) == 0:
    k = np.arange(n/3)
    u = np.stack((fftx3(c[:-2:3]),\
        omega**k * fftx3(c[1:-1:3]),\
        omega**(2*k) * fftx3(c[2::3])))
    F = np.exp(-2j*np.pi/3)** \
        np.array([[0,0,0],[0,1,2],[0,2,4]])    
    return((F @ u).flatten() )
  else:
    k = np.arange(n)[:,None]
    F = omega**(k*k.T);
    return( (F @ c))

In [ ]:
from scipy.fft import fft
v = np.random.rand(24)
np.c_[fft(v),fftx3(v)]

**6.2. Fast multiplication** The following function uses FFTs to multiply two large integers (inputted as strings). We'll verify that the algorithm works by multplying the [RSA-129 factors](https://en.wikipedia.org/wiki/RSA_numbers#RSA-129).  Python uses arbitrary-precision integers, so we can simply multiply the numbers directly.

In [ ]:
def multiply(p_,q_):
  from scipy.signal import fftconvolve
  p = np.flip(np.array([int(i) for i in list(p_)]))
  q = np.flip(np.array([int(i) for i in list(q_)]))
  pq = np.rint(fftconvolve(p,q)).astype(int)
  pq = np.r_[pq,0]
  carry = pq//10
  while (np.any(carry)):
    pq -= carry*10
    pq[1:] += carry[:-1]
    carry = pq//10
  return(''.join([str(i) for i in np.flip(pq)]).lstrip('0'))

In [ ]:
display(32769132993266709549961988190834461413177642967992942539798288533 *\
3490529510847650949147849619903898133417764638493387843990820577)
multiply('32769132993266709549961988190834461413177642967992942539798288533',\
'3490529510847650949147849619903898133417764638493387843990820577')

**6.3. Fast discrete cosine transform**

In [ ]:
from scipy.fft import fft,ifft
def dct(f):
  n = f.shape[0]
  w = np.exp(-0.5j*np.pi*np.arange(n)/n).reshape(-1,1)
  i = [*range(0,n,2),*range(n-1-n%2,0,-2)]
  return(np.real(w*fft(f[i,:],axis=0)))

In [ ]:
def idct(f):
  n = f.shape[0]
  w = np.exp(-0.5j*np.pi*np.arange(n)/n).reshape(-1,1)
  i = [n-(i+1)//2 if i%2 else i//2 for i in range(n)]
  f[0,:] = f[0,:]/2
  return(np.real(ifft(f/w,axis=0))[i,:]*2)

In [ ]:
def dct2(f): return(dct(dct(f.T).T))
def idct2(f): return(idct(idct(f.T).T))

In [ ]:
import urllib
f = urllib.request.urlopen(bucket+'red-fox.jpg')
image = plt.imread(f,format='jpg')
A = np.dot(image[...,:3], [0.2989, 0.5870, 0.1140])
B = dct2(A)[:50,:50]
C = idct2(np.pad(B,((0,A.shape[0]-50),(0,A.shape[1]-50))))
plt.figure(figsize=(8, 6), dpi=160)
plt.imshow(np.c_[A,C], cmap=plt.get_cmap('gray'))
plt.axis('off'); plt.show()

<a name="label18"></a>
## Numerical Analysis
**8.9. Solving a nonlinear system.** We'll solve $$(x^2+y^2)^2 - 2 (x^2 - y^2) =0$$ $$(x^2+y^2 -1)^3-x^2y^3 = 0$$ using homotopy continuation and Newton's method.

In [ ]:
def f(x,y): return(np.array([(x**2+y**2)**2-2*(x**2-y**2),
  (x**2+y**2-1)**3-x**2*y**3]))
def df(x,y): return(np.array([ \
  [4*x*(x**2+y**2-1),  4*y*(x**2+y**2+1)],
  [6*x*(x**2+y**2-1)**2-2*x*y**3, \
   6*y*(x**2+y**2-1)**2-3*x**2*y**2]]))

In [ ]:
def homotopy(f,df,x):
  from scipy.integrate import solve_ivp
  def dxdt(t,x,p): return(la.solve(-df(x[0],x[1]),p))
  sol = solve_ivp(dxdt,[0,1],x,args=(f(x[0],x[1]),))
  return(sol.y[:,-1])

In [ ]:
def newton(f,df,x):
  for i in range(100):
    dx = -la.solve(df(x[0],x[1]),f(x[0],x[1]))
    x += dx
    if (la.norm(dx) < 1e-8): return(x)

In [ ]:
x0 = [-1,-1]
print(homotopy(f,df,x0))
print(newton(f,df,x0))

**9.2. Periodic parametric splines.** We modify the code [`spline_natural`](#spline_natural) (above) to  make a generate a spine with periodic boundary conditions. The function `evaluate_spline` is duplicated from the code above.

In [ ]:
def spline_periodic(x,y):
  h = np.diff(x)
  d = 6*np.diff(np.diff(np.r_[y[-2],y])/np.r_[h[-1],h])
  a = h[:-1]
  b = h + np.r_[h[-1],h[:-1]]
  C = np.diag(b)+np.diag(a,1)
  C[0,-1]=h[-1]; C += C.T 
  m = la.solve(C,d)
  return(np.r_[m,m[0]])

In [ ]:
def evaluate_spline(x,y,m,n):
  h = np.diff(x)
  B = y[:-1] - m[:-1]*h**2/6
  A = np.diff(y)/h-h/6*np.diff(m)
  X = np.linspace(np.min(x),np.max(x),n+1)    
  i = np.array([np.argmin(X>=x)-1 for X in X])
  i[-1] = len(x)-2
  Y = (m[i]*(x[i+1]-X)**3 + m[i+1]*(X-x[i])**3)/(6*h[i]) \
      + A[i]*(X-x[i]) + B[i]
  return(X,Y)

In [ ]:
n, nx = 10, 20
x, y = np.random.rand(n), np.random.rand(n)
x, y = np.r_[x,x[0]], np.r_[y,y[0]]
t = np.cumsum(np.sqrt(np.diff(x)**2+np.diff(y)**2))
t = np.r_[0,t]  
T,X = evaluate_spline(t,x,spline_periodic(t,x),nx*n)
T,Y = evaluate_spline(t,y,spline_periodic(t,y),nx*n)
plt.plot(X,Y,x,y,'o'); plt.show()

**9.3. Radial basis functions.** Let's examine how a polynomial $y(x) = \sum_{i=0}^n c_i x^i$ compares with Gaussian and cubic radial basis functions $y(x) = \sum_{i=0}^n c_i \phi(x-x_i)$ taking $\phi(x)= \exp(-20x^2)$ and $\phi(x) = |x|^3$ an interpolant of the Heaviside function.

In [ ]:
n = 20; N = 200
x = np.linspace(-1,1,n)[:,None]
X = np.linspace(-1,1,N)[:,None]
y = (x>0)

In [ ]:
def phi1(x,a): return(abs(x-a)**3)
def phi2(x,a): return(np.exp(-20*(x-a)**2))
def phi3(x,a): return(x**a)
def interp(phi,a): 
  return(phi(X,a.T)@la.solve(phi(x,a.T),y))

In [ ]:
Y1 = interp(phi1,x)
Y2 = interp(phi2,x)
Y3 = interp(phi3,np.arange(n))
plt.plot(x,y,X,Y1,X,Y2,X,Y3)
plt.ylim((-.5,1.5)); plt.show()

**9.4. Collocation.** We'll use collocation to solve Bessel's equation.  We first define a function to solve general linear boundary value problems. And, then we define a function to interpolate between collocation points.

In [ ]:
def solve(L,f,bc,x):
  h = x[1]-x[0]
  S = np.array([[1,-1/2,1/6],[-2,0,2/3],[1,1/2,1/6]])  \
          /np.array([h**2,h,1])
  S = np.r_[np.zeros((1,3)),L(x)@S.T,np.zeros((1,3))]
  d = np.r_[bc[0], f(x), bc[1]]
  A = np.diag(S[1:,0],-1) + np.diag(S[:,1]) + np.diag(S[:-1,2],1)
  A[0,:3] , A[-1,-3:] = np.array([1,4,1])/6 , np.array([1,4,1])/6
  return(la.solve(A,d))

In [ ]:
def build(c,x,N):
  X = np.linspace(x[0],x[-1],N)
  h = x[1] - x[0]
  i = (X // h).astype(int)
  i[-1] = i[-2]
  C = np.c_[c[i],c[i+1],c[i+2],c[i+3]]
  B = lambda x: np.c_[(1-x)**3, 4-3*(2-x)*x**2, \
           4-3*(1+x)*(1-x)**2, x**3]/6
  Y = np.sum(C*B((X-x[i])/h),axis=1)
  return(X,Y)

Now, we can solve the Bessel equation $xu''+u'+xu =0$ with boundary conditions $u(0)=1$ and $u(b)=0$.

In [ ]:
from scipy.special import jn_zeros, j0
n = 15; N = 141
L = lambda x: np.c_[x,np.ones_like(x),x]
f = lambda x: np.zeros_like(x)
b = jn_zeros(0,4)[-1]
x = np.linspace(0,b,n)
c = solve(L,f,[1,0],x)
X,Y = build(c,x,N)
plt.plot(X,Y,X,j0(X)); plt.show()

Finally, we'll examine the error and convergence rate.

In [ ]:
N = 10*2**np.arange(6)
e = []
for n in N:
  x = np.linspace(0,b,n)
  c = solve(L,f,[1,0],x)
  [X,Y] = build(c,x,n)
  e = np.r_[e,la.norm(Y-j0(X))/n]
plt.loglog(N,e,'.-'); plt.show()

In [ ]:
from numpy.polynomial.polynomial import polyfit 
s = polyfit(np.log(N),np.log(e),1)[1]
print("slope = " + "{:4.4f}".format(s))

**10.3. Fractional derivatives.** We'll plot the fractional derivatives for a function.

In [ ]:
from numpy.fft import fft,ifft,fftshift
def f(x): return np.exp(-16*x**2)
def f(x): return x*(1-np.abs(x))
n = 2000; L = 2 
x = np.arange(n)/n*L-L/2
k = fftshift(np.arange(n)-n/2)*2*np.pi/L

In [ ]:
from ipywidgets import interactive
def anim(derivative=0):
  u = np.real(ifft((1j*k)**derivative*fft(f(x))))
  plt.plot(x,u,color='k'); plt.show()
interactive(anim, derivative = (0,2,0.01))

**10.4. Levenberg–Marquardt method.** We find parameters for a nonlinear function that best fits our data. We'll first define a function `gauss_newton` and duplicate the function [`jacobian`](#jacobian) from above. If the Gauss–Newton method converges, then we can plot the results.

In [ ]:
def gauss_newton(x,y,c,f):
  r = y - f(c,x)
  for j in range(100):
    G = jacobian(f,c,x)
    M = G.T @ G
    c += la.solve(M + np.diag(np.diag(M)),G.T@r)
    r, r0 = y-f(c,x), r
    if la.norm(r-r0) < 10E-8: return(c)
  print('Gauss-Newton did not converge.')

In [ ]:
def jacobian(f,c,x):
  J = np.empty([len(x), len(c)])
  n = np.arange(len(c))
  for i in n:
    J[:,i] = np.imag(f(c+1e-8j*(i==n),x))/1e-8
  return(J)

In [ ]:
def f(c,x): return( c[0]*np.exp(-c[1]*(x-c[2])**2) +
  c[3]*np.exp(-c[4]*(x-c[5])**2))
x = 8*np.random.rand(100)
y = f(np.array([1,3,3,2,3,6]),x) + np.random.normal(0,0.1,100)
c0 = np.array([2,0.3,2,1,0.3,7])
c = gauss_newton(x,y,c0,f)

In [ ]:
X = np.linspace(0,8,100)
plt.plot(x,y,'.',X,f(c,X)); plt.show()

**10.5. Handwriting classification.** We'll use Keras to train a convolutional neural net using MNIST data and then test the model.

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Conv2D, AvgPool2D, Dense, Flatten
model = keras.models.Sequential([
  Conv2D(6,5,activation='tanh',padding='same',input_shape=(28,28,1)),
  AvgPool2D(),
  Conv2D(16, 5, activation='tanh'),
  AvgPool2D(),
  Conv2D(120, 5, activation='tanh'),
  Flatten(),
  Dense(84, activation='tanh'),
  Dense(10, activation='sigmoid')
])

In [ ]:
model.build(); model.summary()

In [ ]:
from keras.datasets import mnist
(image_train,label_train),(image_test,label_test) = mnist.load_data()
image_train = tf.expand_dims(image_train/255.0, 3)
image_test = tf.expand_dims(image_test/255.0, 3)

In [ ]:
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer="adam",  loss=loss, metrics=["accuracy"])

In [ ]:
model.fit(image_train, label_train, epochs=5)

In [ ]:
model.evaluate(image_test,label_test)

**11.1. Finite difference approximation.**  Let's find coefficients to the third-order approximation of $f'(x)$ for nodes at $x$, $x+h$, $x+2h$ and $x+3h$.  We'll reuse the function [`rats`](#rats) from above.

In [ ]:
d = np.array([0,1,2,3])[:,None]; n = len(d)
factorial = [np.math.factorial(i) for i in range(n+1)]
V = d**np.arange(n) / factorial[:-1]
C = la.inv(V)
C = np.c_[C,C@d**n/factorial[-1]]

In [ ]:
from fractions import Fraction
def rats(x): return str(Fraction(x).limit_denominator())

The coefficients of the finite difference approximation of the derivative and coefficient of the truncation error are given by

In [ ]:
print("Coefficients: "+", ".join([rats(x) for x in C[1,:-1]]))
print("Truncation: "+rats(C[1,-1]) )

**11.2. Richardson extrapolation.** The following code is an iterative version of the recursive [`richardson`](#richardson_extrapolation) function above:

In [ ]:
def richardson(f,x,m):
  D = np.zeros(m)
  for i in range(m):
    D[i] = phi(f,x,2**(i+1))
    for j in range(i-1,-1,-1):
      D[j] = (4**(i-j)*D[j+1] - D[j])/(4**(i-j) - 1)
  return(D[1])

**11.3. Automatic differentiation.** Let's extend the [`Dual class`](#dualclass) above by adding methods for division, cosine, and square root to the class definition. We'll also add a few more help functions. 

In [ ]:
class Dual:
  def __init__(self, value, deriv=1):
    self.value = value
    self.deriv = deriv
  def __add__(u, v):
    return Dual(value(u) + value(v), deriv(u) + deriv(v))
  __radd__ = __add__
  def __sub__(u, v):
    return Dual(value(u) - value(v), deriv(u) - deriv(v))
  __rsub__ = __sub__
  def __mul__(u, v):
    return Dual(value(u)*value(v), 
        value(v)*deriv(u) + value(u)*deriv(v))
  __rmul__ = __mul__
  def sin(u):
    return Dual(sin(value(u)),cos(value(u))*deriv(u)) 
  def __truediv__(u, v):
    return Dual(value(u) / value(v), 
      (value(v)*deriv(u)-value(u)*deriv(v))/(value(v)*value(v)))
  __rtruediv__ = __truediv__
  def cos(u):
    return Dual(cos(value(u)),-1*sin(value(u))*deriv(u))
  def sqrt(u):
    return Dual(sqrt(value(u)),deriv(u)/(2*sqrt(value(u))))

In [ ]:
def value(x): 
  return(x.value if isinstance(x, Dual) else x)
def deriv(x): 
  return(x.deriv if isinstance(x, Dual) else 0)
def sin(x): return np.sin(x) 
def cos(x): return np.cos(x) 
def auto_diff(f,x): 
    return f(Dual(x)).deriv
def cos(u): return np.cos(u)
def sqrt(u): return np.sqrt(u)

Now, we'll can define Newton's method using this new Dual class and use it to find the zero of $4\sin x + \sqrt{x}$.

In [ ]:
def get_zero(f,x):
  tolerance = 1e-14; delta = 1
  while abs(delta)>tolerance:
    fx = f(Dual(x))
    delta = value(fx)/deriv(fx)
    x -= delta
  return(x)

In [ ]:
def f(x): return 4*sin(x) + sqrt(x)
get_zero(f, 4)

We can find a minimum or maximum of $4\sin x + \sqrt{x}$ by modifying Newton's method.

In [ ]:
def get_extremum(f,x):
  tolerance = 1e-14; delta = 1
  while abs(delta)>tolerance:
    fx = f(Dual(Dual(x)))
    delta = deriv(value(fx))/deriv(deriv(fx))
    x -= delta
  return(x)

In [ ]:
get_extremum(f, 4)

**11.4. Gauss–Legendre quadrature .** The following  function computes the nodes and weights for  Gauss–Legendre quadrature by using Newton's method to find the roots of $\mathrm{P_n}(x)$. We'll verify the function on a toy problem.

In [ ]:
def gauss_legendre(n):
  x = -np.cos((4*np.arange(n)+3)*np.pi/(4*n+2))
  dx = np.ones_like(x)
  dP = 0
  while(max(abs(dx))>1e-16):
    P0, P1 = x, np.ones_like(x)
    for k in range(2,n+1):
      P0, P1 = ((2*k - 1)*x*P0-(k-1)*P1)/k, P0 
    dP = n*(x*P0 - P1)/(x**2-1)
    dx =  P0 / dP 
    x -= dx
  return( x, 2/((1-x**2)*dP**2) )

In [ ]:
def f(x): return 2*sqrt(1-x**2)
x,w = gauss_legendre(10)
np.dot(w,f(x))

**11.6. Fundamental solution to the heat equation.** We'll use Gauss–Hermite quadrature to compute the solution to the heat equation $$u(t,x) = \frac{1}{\sqrt{4\pi t}}\int_{-\infty}^\infty  u_0(s) \mathrm{e}^{-(x-s)^2/4t} \;\mathrm{d}s.$$

In [ ]:
ξ,w = np.polynomial.hermite.hermgauss(40)
def u0(x): return np.sin(x)
def u(t,x): 
  return [np.dot(w,u0(x-2*np.sqrt(t)*ξ)/np.sqrt(np.pi)) for x in x]
x = np.linspace(-12,12,100)
plt.plot(x,u(1,x)); plt.show()

**11.7. Monte Carlo integration.** The following  function the volume of an $d$-dimensional sphere using $n$ samples and $m$ trials. We'll use it to verify that error of Monte Carlo integretion is $O(1/\sqrt{n})$.

In [ ]:
def mc_pi(n,d,m): 
  return(sum(sum(np.random.rand(d,n,m)**2)<1)/n*2**d)

In [ ]:
m = 20; error = []; N = 2**np.arange(20)
error = [sum(abs(np.pi - mc_pi(n,2,m)))/m for n in N]
plt.loglog(N,error,marker=".",linestyle="None")
s = np.polyfit(np.log(N),np.log(error),1)
plt.loglog(N,np.exp(s[1])*N**s[0])
plt.show()
print("slope = {:4.3f}".format(s[0]))

<a name="label19"></a>
## Numerical Differential Equations
**12.4. Runge–Kutta  stability** The following code plots the region of absolute stability for a Runge–Kutta method with tableau `A` and `b`.

In [ ]:
A = np.array([
  [0,   0,   0,   0,   0],
  [1/3, 0,   0,   0,   0],
  [1/6, 1/6, 0,   0,   0],
  [1/8, 0,   3/8, 0,   0],
  [1/2, 0,  -3/2, 2,   0]])
b = np.array([[1/6, 0, 0, 2/3, 1/6]])

In [ ]:
N = 100; n = b.shape[1]
r = np.zeros((N,N))
E = np.ones((n,1))
x,y = np.linspace(-4,4,N),np.linspace(-4,4,N)
for i in range(N):
  for j in range(N):
    z = x[i] + 1j*y[j]
    r[j,i] = abs(1 + z*b@(la.solve(np.eye(n) - z*A,E)))
plt.contour(x,y,r,[1]); plt.show()

**12.8. Third-order IMEX coefficients.** We can determine the coefficients of a third-order IMEX method by inverting a system of linear equations.

In [ ]:
i = np.arange(4)[:,None]
def factorial(k): return np.cumprod(np.r_[1,range(1,k)])
c1 = la.solve(((-i)**i.T/factorial(4)).T,np.array([0,1,0,0]))
c2 = la.solve(((-(i+1))**i.T/factorial(4)).T,np.array([1,0,0,0]))

In [ ]:
from fractions import Fraction
def rats(x): return str(Fraction(x).limit_denominator())

In [ ]:
print("right-hand side: " + ", ".join([rats(c) for c in c1]))
print("implicit: " + ", ".join([rats(c) for c in c2]))

**12.9. Predictor-corrector stability.** We'll use the  [`multistepcoeffs`](#multistepcoeffs) introduced earlier. The following function provides the orbit of points in the complex plane for an $n$th order  Adams–Bashforth–Moulton PE(CE)$^m$.

In [ ]:
def multistepcoeffs(m,n):
  s = len(m) + len(n)
  A = (np.r_[m]+1.0)**np.c_[range(s)]
  B = np.c_[range(s)]*(np.r_[n]+1.0)**(np.c_[range(s)]-1)
  c = la.solve(-np.c_[A,B],np.ones((s,1))).flatten()
  return(np.r_[1,c[:len(m)]], c[len(m):] )

In [ ]:
def PECE(n,m):
  _,a = multistepcoeffs([1],range(1,n+1))
  _,b = multistepcoeffs([1],range(0,n+1))
  def c(r): return np.r_[r-1,\
    np.full(m, r + (b[1:] @ r**np.arange(1,n+1))/b[0]),\
    (a @ r**np.arange(1,n+1))/b[0]]
  z = []
  for r in np.exp(1j*np.linspace(0,2*np.pi,200)):
    z = np.append(z,np.roots(np.flip(c(r)))/b[0])
  return z

In [ ]:
for i in range(5): 
  z = PECE(4,i)      
  plt.scatter(np.real(z),np.imag(z),s=0.5)
plt.axis('equal'); plt.show()

**12.10. Padé approximant.** We'll build a function to compute the coefficients of the Padé approximant to $log(r)$.

In [ ]:
def pade(a,m,n):
  A = np.eye(m+n+1);
  for i in range(n): A[i+1:,m+i+1] = -a[:m+n-i]
  pq = la.solve(A,a[:m+n+1])
  return pq[:m+1], np.r_[1,pq[m+1:]]

In [ ]:
m = 3; n = 2
a = np.r_[0, (-1)**np.arange(m+n)/(1+np.arange(m+n))]
p,q = pade(a,m,n)

In [ ]:
def S(n): return la.invpascal(n+1, kind='upper')
S(m)@p, S(n)@q

**12.14. SIR solution.** We solve the susceptible-infected-recovered (SIR) model for infectious diseases using a general ODE solver.

In [ ]:
from scipy.integrate import solve_ivp
def SIR(t,y,b,g): return (-b*y[0]*y[1],b*y[0]*y[1]-g*y[1],g*y[1])
sol = solve_ivp(SIR, [0, 15], [0.99, 0.01, 0],\
  args=(2,0.4), dense_output=True)
t = np.linspace(0,15,200); y = sol.sol(t)
plt.plot(t,y[0,:],t,y[1,:],t,y[2,:]); plt.show()

**12.15. Duffing equation.** We'll use a high-order, explicit ODE solver for the Duffing equation.

In [ ]:
from scipy.integrate import solve_ivp
def duff(t,x,g): return(x[1],-g*x[1]+x[0]-x[0]**3+0.3*np.cos(t))
sol = solve_ivp(duff,[0,200], [1, 0], args=(0.37,), 
    method='DOP853',dense_output=True)
t = np.linspace(0,200,2000); y = sol.sol(t)
plt.plot(y[0,:],y[1,:]); plt.show()

**12.16. Shooting method.** We'll solve the Airy equation $y'' - xy = 0$ using the shooting method that incorporates an initial value solver along into a nonlinear root finder.

In [ ]:
from scipy.integrate import solve_ivp
from scipy.optimize import fsolve
def airy(x,y): return(y[1],x*y[0])
domain = [-12,0]; bc = [1,1]; guess = 5
def shoot_airy(guess):
  sol = solve_ivp(airy,domain,[bc[0],guess[0]])
  return sol.y[0,-1] - bc[1] 
v = fsolve(shoot_airy,guess)[0]

Once we have our second initial value, we can plot the solution:

In [ ]:
sol = solve_ivp(airy,domain,[bc[0],v],dense_output=True)
x = np.linspace(-12,0,200)
plt.plot(x,sol.sol(x)[0,:]); plt.show()

**13.4. Dufort–Frankel method.** We'll use the Dufort–Frankel method to solve the heat equation. While this method is unconditionally stable, it generates the wrong solution. Notice that while the long-term behavior is dissipative, the solution is largely oscillatory and the dynamics are more characteristic of a viscous fluid than heat propagation.

In [ ]:
dx = 0.01; dt = 0.01; n = 400
L = 1; x = np.arange(-L,L,dx); m = len(x) 
U = np.empty((n,m))
U[0,:] = np.exp(-8*x**2); U[1,:] = U[0,:]  
c = dt/dx**2; a = 0.5 + c; b = 0.5 - c
start_time = timeit.default_timer()
B = c*sp.diags([1, 1], [-1, 1], shape=(m, m)).tocsr()
B[0,1] *=2; B[-1,-2] *=2
for i in range(1,n-1):
  U[i+1,:] = (B@U[i,:]+b*U[i-1,:])/a

In [ ]:
from ipywidgets import interactive
def anim(i=0):
  plt.fill_between(x,U[i,:],color='#ff9999');
  plt.ylim(0,1);plt.show()
interactive(anim, i = (0,n-1))

**13.6. Schrödinger equation.** We solve the Schrödinger equation for harmonic potential using the Strang splitting Crank-Nicolson and confirm that the method is $O(h^2,k^2)$.

In [ ]:
def psi0(x,eps): return np.exp(-(x-1)**2/(2*eps))/(np.pi*eps)**(1/4)
def schroedinger(n,m,eps):
  x,dx = np.linspace(-4,4,n,retstep=True); dt = 2*np.pi/m; V = x**2/2
  psi = psi0(x,eps)
  D = 0.5j*eps*sp.diags([1, -2, 1], [-1, 0, 1], shape=(n, n))/dx**2 \
    - 1j/eps*sp.diags(V,0)
  D[0,1] *= 2; D[-1,-2] *= 2
  A = sp.eye(n) + (dt/2)*D 
  B = sp.eye(n) - (dt/2)*D  
  for i in range(m):
    psi = sp.linalg.spsolve(B,A*psi)
  return(psi)

We'll loop over several values for time steps and mesh sizes and plot the error. This may take a while. Go get a snack.

In [ ]:
eps = 0.3; m = 20000; N = np.logspace(2,3.7,6).astype(int)
x = np.linspace(-4,4,m)
psi_m = -psi0(x,eps)
error_t = []; error_x = []
for n in N: 
  x = np.linspace(-4,4,n) 
  psi_n = -psi0(x,eps)
  error_t.append(la.norm(psi_m - schroedinger(m,n,eps))/m)
  error_x.append(la.norm(psi_n - schroedinger(n,m,eps))/n)
plt.loglog(2*np.pi/N,error_t,'.-r',8/N,error_x,'.-k'); plt.show()

**13.7. Polar heat equation.** We'll solve a radially symmetric heat equation. Although we divide by zero at $r=0$ when constructing the Laplacian operator, the resulting term `inf` is overwritten when we apply the boundary condition.

In [ ]:
T = 0.5; m = 100; n = 100
r = np.linspace(0,2,m); dr = r[1]-r[0]; dt = T/n
u = np.tanh(32*(1-r))[:,None]
D = sp.diags([1, -2, 1], [-1, 0, 1], shape=(m,m))/dr**2 \
  + sp.diags([-1/r[1:], 1/r[:-1]], [-1, 1])/(2*dr)
D[0,0:2] = np.array([-4,4])/dr**2;
D[-1,-2:] = np.array([2,-2])/dr**2 
A = sp.eye(m) - 0.5*dt*D 
B = sp.eye(m) + 0.5*dt*D  
for i in range(n):
  u = sp.linalg.spsolve(A,B@u)
plt.fill_between(r,u,-1,color='#ff9999'); plt.show()

**13.8. Open boundaries.** We can approximate open boundaries by spacing the grid points using a sigmoid function such as $\mathrm{arctanh}\, x$. We start by defining a function `logitspace` which is the logit analogue to `np.linspace`. Then we define a Laplacian operator using arbitrary grid spacing. Finally, we solve the heat equation using the Crank–Nicolson using both equally spaced gridpoints and logit spaced gridpoints.

In [ ]:
def logitspace(x,n,p): 
  return x*np.arctanh(np.linspace(-p,p,n))/np.arctanh(p)

In [ ]:
def laplacian(x):
  h = np.diff(x); h1 = h[:-1]; h2 = h[1:]; n = len(x)
  d = np.c_[ \
    np.r_[h1[0]**2, h2*(h1+h2),0], \
    np.r_[-h1[0]**2, -h1*h2,-h2[-1]**2 ], \
    np.r_[h1*(h1+h2), h2[-1]**2,0]].T
  d[0,-1],d[2,-1] = 999,999
  return sp.diags(2/d,[-1,0,1],shape=(n, n)).T

In [ ]:
def heat_equation(x,t,u):
  m = 40; dt = t/m
  u = phi(x,0,10)
  D = laplacian(x)
  A = sp.eye(len(x)) - 0.5*dt*D
  B = sp.eye(len(x)) + 0.5*dt*D
  for i in range(m):
    u = sp.linalg.spsolve(A,B@u)
  return(u)

In [ ]:
def phi(x,t,s): 
  return np.exp(-s*x**2/(1+4*s*t))/np.sqrt(1+4*s*t)
t = 15; n = 40
x = logitspace(20,n,.999)
laplacian(x).toarray()
u = heat_equation(x,t,phi(x,0,10))
plt.plot(x,u,'.-',x,phi(x,t,10),'k'); plt.show()
x = np.linspace(-20,20,n)
u = heat_equation(x,t,phi(x,0,10))
plt.plot(x,u,'.-',x,phi(x,t,10),'k'); plt.show()

**13.9. Allen–Cahn equation.** We'll solve the Allen–Cahn equation using  Strang splitting. We'll save the solution every tenth iteration and animate the solution using the `ipywidgets` library.  Rerun the code by uncommenting the random initial conditions.

In [ ]:
L = 16; m = 200; dx = L/m
T = 8; n = 1600; dt = T/n
x = np.linspace(-L/2,L/2,m)[None,:]
u = np.tanh(x**4 - 16*(2*x**2-x.T**2))
#u = np.random.standard_normal((m,m))
D = sp.diags([1, -2, 1], [-1, 0, 1], shape=(m,m)).tocsr()/dx**2
D[0,1] *= 2; D[-1,-2] *= 2;
A = sp.eye(m) + 0.5*dt*D
B = sp.eye(m) - 0.5*dt*D 
def f(u,dt): 
  return u/np.sqrt(u**2 - (u**2-1)*np.exp(-50*dt))
u = f(u,dt/2)
U = np.empty((m,m,n//10))
for i in range(n):
  if (i%8==0): U[:,:,i//10] = u
  u = sp.linalg.spsolve(B,(A@sp.linalg.spsolve(B,A@u).T)).T
  if (i<n): u = f(u,dt)
u = f(u,dt/2)

In [ ]:
from ipywidgets import interactive
def anim(i=0):
  plt.imshow(U[:,:,i], cmap="gray"); plt.axis('off'); plt.show()
interactive(anim, i = (0,U.shape[2]-1))

**14.7. Burgers' Equation.**

In [ ]:
m = 100; x,dx = np.linspace(-1,3,m,retstep=True)
n = 100; Lt = 4; dt = Lt/n; c = dt/dx
def f(u): return u**2/2
def fp(u): return u
u = ((x>=0)&(x<=1)).astype(float)
for i in range(n):
  fu = f(np.r_[u[0],u]); fpu = fp(np.r_[u[0],u])
  a = np.maximum(abs(fu[1:-1]),abs(fu[:-2]))
  F = (fu[1:-1]+fu[:-2])/2 - a*(u[1:]-u[:-1])/2
  u -= c*(np.diff(np.r_[0,F,0]))

**14.8. Dam break problem.**

In [ ]:
def limiter(t): return (abs(t)+t)/(1+abs(t))  
def fixzero(u): return u + (u==0).astype(float)
def diff(u): return np.diff(u,axis=0)
def slope(u):
  du = diff(u)
  return np.r_[np.c_[0,0], \
    np.c_[du[1:,:]*limiter(du[:-1,:]/fixzero(du[1:,:]))],\
    np.c_[0,0]]
def F(u):
  return np.c_[u[:,0]*u[:,1], u[:,0]*u[:,1]**2+0.5*u[:,0]**2]

In [ ]:
m = 1000; x,dx = np.linspace(-.5,.5,m,retstep=True)
T = 0.25; n = (T/(dx/2)).astype(int); dt = (T/n)/2; c = dt/dx
u = np.c_[0.8*(x<0)+0.2,0*x] 
for i in range(n):
  v = u-0.5*c*slope(F(u))
  u[1:,:]=(u[:-1,:]+u[1:,:])/2 - diff(slope(u))/8 - c*diff(F(v)) 
  v = u-0.5*c*slope(F(u))
  u[:-1,:]=(u[:-1,:]+u[1:,:])/2 - diff(slope(u))/8 - c*diff(F(v))
plt.plot(x,u); plt.show()

**15.1. Finite element method.**

In [ ]:
m = 10; x,h = np.linspace(0,1,m,retstep=True)
A = np.tile(np.r_[-1/h-h/6,2/h-2/3*h,-1/h-h/6],(m,1)).T
A[1,0]/=2; A[1,-1] /= 2
b=np.r_[-2/3*h**3,-4/3*h**3-8*h*x[1:-1]**2,-4*h+8/3*h**2-2/3*h**3+1]
u=la.solve_banded((1,1),A,b)
s=(-16)+8*x**2+15*np.cos(x)/np.sin(1)
plt.plot(x,s,'o-',x,u,'.-'); plt.show()

**15.2. Finite element method.**

In [ ]:
m = 8; x,h = np.linspace(0,1,m+2,retstep=True)
def tridiag(a,b,c): return np.diag(a,-1)+np.diag(b,0)+np.diag(c,1)
def D(a,b,c):   
  return tridiag(a*np.ones(m-1), b*np.ones(m), c*np.ones(m-1))/h**3
M = np.r_[np.c_[D(-12,24,-12),D(-6,0,6)],
  np.c_[D(6,0,-6),D(2,8,2)]]
b = np.r_[np.ones(m)*h*384,np.zeros(m)]
u = la.solve(M,b)
plt.plot(x,16*(x**4 - 2*x**3 + x**2),'o-',x,np.r_[0,u[:m],0],'.-') 
plt.show()

**16.2. Burgers' equation.** Fourier spectral methods perform poorly on problems that develop discontinuities such as Burgers' equation.  Gibbs oscillations develop around the discontinuity, and these oscillations will spread and grow because Burgers' equation is dispersive. Ultimately, the oscillations overwhelm the solution.

In [ ]:
from scipy.integrate import solve_ivp
from scipy.fft import fftshift, fft, ifft
m = 128; x = np.linspace(-np.pi,np.pi,m,endpoint=False)
k = 1j*fftshift(np.arange(-m/2,m/2))
def f(t,u): return -np.real(ifft(k*fft(0.5*u**2)))
sol = solve_ivp(f, [0,1.5], np.exp(-x**2), method = 'RK45')  
plt.plot(x,sol.y[:,-1]); plt.show()

**16.3. KdV equation.** We'll solve the KdV equation using integrating factors. We first set the initial conditions and parameters. Then, we define the integrating factor `G` and the right-hand side `f` of the differential equation. Finally, we animate the solution. Notice the two soliton solution.

In [ ]:
from scipy.fft import fftshift, fft, ifft
def phi(x,x0,c): return 0.5*c/np.cosh(np.sqrt(c)/2*(x-x0))**2
L = 30; T = 2.0; m = 256
x = np.linspace(-L/2,L/2,m,endpoint=False)
k = 1j*fftshift(np.arange(-m/2,m/2))*(2*np.pi/L)

In [ ]:
def G(t): return np.exp(-k**3*t)
def f(t,w): return -(3*k*fft(ifft(G(t)*w)**2))/G(t)

In [ ]:
from scipy.integrate import solve_ivp
u = phi(x,-4,4) + phi(x,-9,9)
w = fft(u)
sol = solve_ivp(f,[0,T],w,t_eval=np.linspace(0,T,200))   
u = [np.real(ifft(G(sol.t[i])*sol.y[:,i])) for i in range(200)]

In [ ]:
plt.rcParams["animation.html"] = "jshtml"
from matplotlib.animation import FuncAnimation
fig, ax = plt.subplots()
l, = ax.plot([-15,15],[0,5])
def animate(i): l.set_data(x, u[i])
FuncAnimation(fig, animate, frames=len(u),interval=50)

**16.4. Swift–Hohenberg equation.** We'll use Strang splitting to solve the  Swift–Hohenberg equation.

In [ ]:
from scipy.fft import fftshift, fft2, ifft2
eps = 1; m = 256; L = 100; n = 2000; dt=100/n
U = (np.random.rand(m,m)>0.5)-0.5
k = fftshift(np.arange(-m/2,m/2))*(2*np.pi/L)
D2 = -k[:,None]**2-k[None,:]**2
E = np.exp(-(D2+1)**2*dt)
def f(U):
  return U/np.sqrt(U**2/eps + np.exp(-dt*eps)*(1-U**2/eps))
for i in range(n):
  U = f(ifft2(E*fft2(f(U))))
plt.imshow(np.real(U), cmap="gray"); plt.axis('off'); plt.show()